In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
np.random.seed(42)

# Adult Dataset Loading

In [3]:
import pandas as pd
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder, OrdinalEncoder

COLUMN_NAMES = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
    'hours-per-week', 'native-country', 'income']
raw_data = pd.read_csv("../data/adult/adult.data", names=COLUMN_NAMES, skipinitialspace=True)
print(raw_data.head())
# one_hot = ["workclass", "education", "relationship", "marital-status", "occupation", "relationship", "race", "sex"]
# one_hot = ["race", "sex"]
subset = ["age", "education-num", "race", "sex", "income", "marital-status", "occupation", "workclass"]
raw_data = raw_data[subset]
raw_data = pd.get_dummies(raw_data, columns=[
    "workclass", "marital-status", "occupation",
])
binarizer = LabelBinarizer(neg_label=0, pos_label=1)
raw_data["income"] = binarizer.fit_transform(raw_data["income"])
raw_data["sex"] = binarizer.fit_transform(raw_data["sex"])
raw_data["race"] = raw_data["race"].map(lambda v: 1 if v == "White" else 0)
# raw_data["marital-status"] = raw_data["marital-status"].map(lambda v: 1 if v == "Never-married" else 0)
# raw_data["relationship"] = raw_data["relationship"].map(lambda v: 1 if v == "Wife" else 0)
labels = raw_data["income"]
raw_data = raw_data.fillna(0)
print(raw_data.isnull().values.any())
# subset.remove("income")
# raw_data = raw_data[subset]
raw_data.columns

   age         workclass  fnlwgt  education  education-num  \
0   39         State-gov   77516  Bachelors             13   
1   50  Self-emp-not-inc   83311  Bachelors             13   
2   38           Private  215646    HS-grad              9   
3   53           Private  234721       11th              7   
4   28           Private  338409  Bachelors             13   

       marital-status         occupation   relationship   race     sex  \
0       Never-married       Adm-clerical  Not-in-family  White    Male   
1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
2            Divorced  Handlers-cleaners  Not-in-family  White    Male   
3  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
4  Married-civ-spouse     Prof-specialty           Wife  Black  Female   

   capital-gain  capital-loss  hours-per-week native-country income  
0          2174             0              40  United-States  <=50K  
1             0             0             

Index(['age', 'education-num', 'race', 'sex', 'income', 'workclass_?',
       'workclass_Federal-gov', 'workclass_Local-gov',
       'workclass_Never-worked', 'workclass_Private', 'workclass_Self-emp-inc',
       'workclass_Self-emp-not-inc', 'workclass_State-gov',
       'workclass_Without-pay', 'marital-status_Divorced',
       'marital-status_Married-AF-spouse', 'marital-status_Married-civ-spouse',
       'marital-status_Married-spouse-absent', 'marital-status_Never-married',
       'marital-status_Separated', 'marital-status_Widowed', 'occupation_?',
       'occupation_Adm-clerical', 'occupation_Armed-Forces',
       'occupation_Craft-repair', 'occupation_Exec-managerial',
       'occupation_Farming-fishing', 'occupation_Handlers-cleaners',
       'occupation_Machine-op-inspct', 'occupation_Other-service',
       'occupation_Priv-house-serv', 'occupation_Prof-specialty',
       'occupation_Protective-serv', 'occupation_Sales',
       'occupation_Tech-support', 'occupation_Transport

In [4]:
adult_raw_data = raw_data
adult_targets = raw_data["income"]

In [5]:
import numpy as np
from sklearn.preprocessing import MaxAbsScaler

# Split into occupation of sales and farming/fishing
# source_data = raw_data[raw_data["sex"] == 1]
# target_data = raw_data[raw_data["sex"] == 0]
adult_source_data = raw_data[raw_data["income"] == 1]
adult_target_data = raw_data[raw_data["income"] == 0]
print(adult_source_data.isnull().values.any())
samples = min(adult_source_data.shape[0], adult_target_data.shape[0])
print(samples)
adult_source_data = adult_source_data.sample(samples, replace=False)
adult_target_data = adult_target_data.sample(samples, replace=False)
print(adult_source_data.isnull().values.any())
adult_source = adult_source_data.to_numpy().astype(float)[:, [i for i in range(adult_source_data.shape[1]) if i != 4]]
adult_target = adult_target_data.to_numpy().astype(float)[:, [i for i in range(adult_source_data.shape[1]) if i != 4]]
adult_feature_names = [adult_source_data.columns[i] for i in range(adult_source_data.shape[1]) if i != 4]
adult_source_labels = adult_source_data.to_numpy().astype(int)[:, 4:]
adult_target_labels = adult_target_data.to_numpy().astype(int)[:, 4:]
adult_feasible_names = [
    # "feasible marital status",

    # "<=10 years",
    # "10-20 years",
    # "20-30 years",
    # "30-40 years",
    # "40-50 years",
    # ">50 years"

    "male",
    "female"

    # "6th-8th grade",
    # "8th-10th grade",
    # "10th-12th grade",
    # "white",
    # "black"
]
adult_source_feasible_groups = np.concatenate([
    # np.logical_or(np.sum(adult_source[:, 14:18], axis=1) > 0, np.sum(
    #     adult_source[:, 19:21], axis=1) > 0).astype(int)[:, np.newaxis],

    # (adult_source[:, 0] <= 10).astype(int)[:, np.newaxis],
    # np.logical_and(adult_source[:, 0] > 10, adult_source[:, 0] <= 20).astype(int)[:, np.newaxis],
    # np.logical_and(adult_source[:, 0] > 20, adult_source[:, 0] <= 30).astype(int)[:, np.newaxis],
    # np.logical_and(adult_source[:, 0] > 30, adult_source[:, 0] <= 40).astype(int)[:, np.newaxis],
    # np.logical_and(adult_source[:, 0] > 40, adult_source[:, 0] <= 50).astype(int)[:, np.newaxis],
    # (adult_source[:, 0] > 50).astype(int)[:, np.newaxis],

    # np.logical_and(adult_source[:, 1] > 6, adult_source[:, 1] <= 8).astype(int)[:, np.newaxis],
    # np.logical_and(adult_source[:, 1] > 8, adult_source[:, 1] <= 10).astype(int)[:, np.newaxis],
    # np.logical_and(adult_source[:, 1] > 10, adult_source[:, 1] <= 12).astype(int)[:, np.newaxis],
    # (adult_source[:, 2] == 1).astype(int)[:, np.newaxis],
    # (adult_source[:, 2] == 0).astype(int)[:, np.newaxis],
    (adult_source[:, 3] == 1).astype(int)[:, np.newaxis],
    (adult_source[:, 3] == 0).astype(int)[:, np.newaxis],
], axis=1)
adult_target_feasible_groups = np.concatenate([
    # np.logical_or(np.sum(adult_target[:, 14:18], axis=1) > 0, np.sum(
    #     adult_target[:, 19:21], axis=1) > 0).astype(int)[:, np.newaxis],

    # (adult_target[:, 0] <= 10).astype(int)[:, np.newaxis],
    # np.logical_and(adult_target[:, 0] > 10, adult_target[:, 0] <= 20).astype(int)[:, np.newaxis],
    # np.logical_and(adult_target[:, 0] > 20, adult_target[:, 0] <= 30).astype(int)[:, np.newaxis],
    # np.logical_and(adult_target[:, 0] > 30, adult_target[:, 0] <= 40).astype(int)[:, np.newaxis],
    # np.logical_and(adult_target[:, 0] > 40, adult_target[:, 0] <= 50).astype(int)[:, np.newaxis],
    # (adult_target[:, 0] > 50).astype(int)[:, np.newaxis],

    # np.logical_and(adult_target[:, 1] > 6, adult_target[:, 1] <= 8).astype(int)[:, np.newaxis],
    # np.logical_and(adult_target[:, 1] > 8, adult_target[:, 1] <= 10).astype(int)[:, np.newaxis],
    # np.logical_and(adult_target[:, 1] > 10, adult_target[:, 1] <= 12).astype(int)[:, np.newaxis],
    # (adult_target[:, 2] == 1).astype(int)[:, np.newaxis],
    # (adult_target[:, 2] == 0).astype(int)[:, np.newaxis],
    (adult_target[:, 3] == 1).astype(int)[:, np.newaxis],
    (adult_target[:, 3] == 0).astype(int)[:, np.newaxis],
], axis=1)
# source = source[:, :5]
# target = target[:, :5]
adult_labels = np.concatenate([adult_source_labels, adult_target_labels])

adult_scaler = MaxAbsScaler().fit(np.concatenate([adult_source, adult_target]))
adult_source = adult_scaler.transform(adult_source)
adult_target = adult_scaler.transform(adult_target)


False
7841
False


In [6]:
adult_raw_data.head()

,age,education-num,race,sex,income,workclass_?,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,occupation_Farming-fishing,occupation_Handlers-cleaners,occupation_Machine-op-inspct,occupation_Other-service,occupation_Priv-house-serv,occupation_Prof-specialty,occupation_Protective-serv,occupation_Sales,occupation_Tech-support,occupation_Transport-moving
0,39,13,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,50,13,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,38,9,1,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
3,53,7,0,1,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
4,28,13,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0


In [7]:
from sklearn.ensemble import GradientBoostingClassifier

X_train = np.concatenate([adult_source, adult_target])
y_train = np.concatenate([adult_source_labels[:, 0], adult_target_labels[:, 0]])
clf = GradientBoostingClassifier(n_estimators=100).fit(X_train, y_train)

adult_source_pred = clf.predict_proba(adult_source)[:, 1]
adult_target_pred = clf.predict_proba(adult_target)[:, 0]
print(adult_source_pred)
print(adult_target_pred)
adult_source_clf_labels = np.concatenate([
    (adult_source_pred < 0.3).astype(int)[:, np.newaxis],
    (adult_source_pred < 0.5).astype(int)[:, np.newaxis],
    (adult_source_pred < 0.7).astype(int)[:, np.newaxis],
    (adult_source_pred < 0.9).astype(int)[:, np.newaxis],
], axis=1)
adult_target_clf_labels = np.concatenate([
    (adult_target_pred < 0.3).astype(int)[:, np.newaxis],
    (adult_target_pred < 0.5).astype(int)[:, np.newaxis],
    (adult_target_pred < 0.7).astype(int)[:, np.newaxis],
    (adult_target_pred < 0.9).astype(int)[:, np.newaxis],
], axis=1)

[0.63297423 0.93361709 0.82191277 ... 0.9422657  0.76794067 0.68708524]
[0.96069158 0.34997801 0.94711941 ... 0.97576716 0.81850217 0.93851661]


In [8]:
def is_adult_feasible_change(orig, new):
    orig_sex = (orig[:, 3].round(0))[:, np.newaxis]
    new_sex = (new[:, 3].round(0))[:, np.newaxis]

    return (
        np.all(new_sex == orig_sex, axis=1) )

In [9]:
adult_source_data.columns

Index(['age', 'education-num', 'race', 'sex', 'income', 'workclass_?',
       'workclass_Federal-gov', 'workclass_Local-gov',
       'workclass_Never-worked', 'workclass_Private', 'workclass_Self-emp-inc',
       'workclass_Self-emp-not-inc', 'workclass_State-gov',
       'workclass_Without-pay', 'marital-status_Divorced',
       'marital-status_Married-AF-spouse', 'marital-status_Married-civ-spouse',
       'marital-status_Married-spouse-absent', 'marital-status_Never-married',
       'marital-status_Separated', 'marital-status_Widowed', 'occupation_?',
       'occupation_Adm-clerical', 'occupation_Armed-Forces',
       'occupation_Craft-repair', 'occupation_Exec-managerial',
       'occupation_Farming-fishing', 'occupation_Handlers-cleaners',
       'occupation_Machine-op-inspct', 'occupation_Other-service',
       'occupation_Priv-house-serv', 'occupation_Prof-specialty',
       'occupation_Protective-serv', 'occupation_Sales',
       'occupation_Tech-support', 'occupation_Transport

# Breast Cancer Data Loading

In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelBinarizer, OneHotEncoder, OrdinalEncoder
from sklearn import preprocessing

COLUMN_NAMES = [
    "diagnosis", "radius_mean","texture_mean","perimeter_mean","area_mean","smoothness_mean","compactness_mean","concavity_mean","concave points_mean","symmetry_mean","fractal_dimension_mean"
    ]
raw_data = pd.read_csv(
    # "../data/breast_cancer/breast-cancer-wisconsin.data",
    "../data/breast_cancer/data.csv",
    # header=None,
    index_col=0)
# raw_data.columns = COLUMN_NAMES
raw_data.drop(raw_data[(raw_data == '?').sum(axis=1) != 0].index, inplace=True)
raw_data = raw_data[COLUMN_NAMES].dropna()

bc_source_df = raw_data.query('diagnosis == "B"').sample(212)
bc_target_df = raw_data.query('diagnosis == "M"').sample(212)
bc_source = bc_source_df.drop(columns='diagnosis').to_numpy().astype(float)
bc_target = bc_target_df.drop(columns='diagnosis').to_numpy().astype(float)

bc_scaler = preprocessing.StandardScaler().fit(np.concatenate([bc_source, bc_target]))
bc_source = bc_scaler.transform(bc_source)
bc_target = bc_scaler.transform(bc_target)

bc_source_labels = np.concatenate([
    (bc_source[:, i] >= np.percentile(bc_source[:, i], 75)).astype(int)[:, np.newaxis] for i in range(1, bc_source.shape[1])],
    axis=1)
bc_target_labels = np.concatenate([
    (bc_target[:, i] >= np.percentile(bc_target[:, i], 75)).astype(int)[:, np.newaxis] for i in range(1, bc_source.shape[1])],
    axis=1)
bc_feature_names = COLUMN_NAMES[1:]
bc_feasible_names = ["radius**2 / area above third quartile", "radius**2 / area between first and third quartile", "radius**2 / area below first quartile",]
#  "radius / perimeter above third quartile", "radius / perimeter below first quartile"]
bc_source_feasible_groups = np.concatenate([
    (((bc_source[:, 0]**2) / bc_source[:, 3]) > np.percentile(((bc_source[:, 0]**2) / bc_source[:, 3]), 75)).astype(int)[:, np.newaxis],
    (
        (((bc_source[:, 0]**2) / bc_source[:, 3]) >= np.percentile(((bc_source[:, 0]**2) / bc_source[:, 3]), 25)) &
        (((bc_source[:, 0]**2) / bc_source[:, 3]) <= np.percentile(((bc_source[:, 0]**2) / bc_source[:, 3]), 75))
    ).astype(int)[:, np.newaxis],
    (((bc_source[:, 0]**2) / bc_source[:, 3]) < np.percentile(((bc_source[:, 0]**2) / bc_source[:, 3]), 25)).astype(int)[:, np.newaxis],
    # (((bc_source[:, 0]) / bc_source[:, 2]) > np.percentile(((bc_source[:, 0]) / bc_source[:, 2]), 75)).astype(int)[:, np.newaxis],
    # (((bc_source[:, 0]) / bc_source[:, 2]) < np.percentile(((bc_source[:, 0]) / bc_source[:, 2]), 25)).astype(int)[:, np.newaxis],
], axis=1)
bc_target_feasible_groups = np.concatenate([
    (((bc_target[:, 0]**2) / bc_target[:, 3]) > np.percentile(((bc_target[:, 0]**2) / bc_target[:, 3]), 75)).astype(int)[:, np.newaxis],
    (
        (((bc_target[:, 0]**2) / bc_target[:, 3]) >= np.percentile(((bc_target[:, 0]**2) / bc_target[:, 3]), 25)) &
        (((bc_target[:, 0]**2) / bc_target[:, 3]) <= np.percentile(((bc_target[:, 0]**2) / bc_target[:, 3]), 75))
    ).astype(int)[:, np.newaxis],
    (((bc_target[:, 0]**2) / bc_target[:, 3]) < np.percentile(((bc_target[:, 0]**2) / bc_target[:, 3]), 25)).astype(int)[:, np.newaxis],
    # (((bc_target[:, 0]) / bc_target[:, 2]) > np.percentile(((bc_target[:, 0]) / bc_target[:, 2]), 75)).astype(int)[:, np.newaxis],
    # (((bc_target[:, 0]) / bc_target[:, 2]) < np.percentile(((bc_target[:, 0]) / bc_target[:, 2]), 25)).astype(int)[:, np.newaxis],
], axis=1)

In [11]:
from sklearn.ensemble import GradientBoostingClassifier

X_train = np.concatenate([bc_source, bc_target])
y_train = np.concatenate([np.ones(bc_source.shape[0]), np.zeros(bc_target.shape[0])])
clf = GradientBoostingClassifier(n_estimators=100).fit(X_train, y_train)

bc_source_pred = clf.predict_proba(bc_source)[:, 1]
bc_target_pred = clf.predict_proba(bc_target)[:, 0]
print(np.sum(bc_source_pred < 0.996), np.sum(bc_target_pred < 0.996))
bc_source_clf_labels = np.concatenate([
    (bc_source_pred < 0.995).astype(int)[:, np.newaxis],
    (bc_source_pred < 0.997).astype(int)[:, np.newaxis],
    (bc_source_pred < 0.999).astype(int)[:, np.newaxis],
    (bc_source_pred < 0.9995).astype(int)[:, np.newaxis],
], axis=1)
bc_target_clf_labels = np.concatenate([
    (bc_target_pred < 0.995).astype(int)[:, np.newaxis],
    (bc_target_pred < 0.997).astype(int)[:, np.newaxis],
    (bc_target_pred < 0.999).astype(int)[:, np.newaxis],
    (bc_target_pred < 0.9995).astype(int)[:, np.newaxis],
], axis=1)

117 92


In [12]:
def is_bc_feasible_change(orig, new):
    ratio1_orig_g1 = (orig[:, 0]**2) / orig[:, 3] > np.percentile(((bc_source[:, 0]**2) / bc_source[:, 3]), 75)
    ratio1_orig_g2 = (orig[:, 0]**2) / orig[:, 3] < np.percentile(((bc_source[:, 0]**2) / bc_source[:, 3]), 25)
    ratio1_orig_g3 = ~ratio1_orig_g1 & ~ratio1_orig_g2
    ratio1_new_g1 = (new[:, 0]**2) / new[:, 3] > np.percentile(((bc_target[:, 0]**2) / bc_target[:, 3]), 75)
    ratio1_new_g2 = (new[:, 0]**2) / new[:, 3] < np.percentile(((bc_target[:, 0]**2) / bc_target[:, 3]), 25)
    ratio1_new_g3 = ~ratio1_new_g1 & ~ratio1_new_g2
    # ratio2_orig_g1 = orig[:, 0] / orig[:, 2] > np.percentile(((bc_source[:, 0]) / bc_source[:, 2]), 75)
    # ratio2_orig_g2 = orig[:, 0] / orig[:, 2] < np.percentile(((bc_source[:, 0]) / bc_source[:, 2]), 25)
    # ratio2_new_g1 = new[:, 0] / new[:, 2] > np.percentile(((bc_target[:, 0]) / bc_target[:, 2]), 75)
    # ratio2_new_g2 = new[:, 0] / new[:, 2] < np.percentile(((bc_target[:, 0]) / bc_target[:, 2]), 25)
    return ((ratio1_orig_g1 == ratio1_new_g1) &
        (ratio1_orig_g2 == ratio1_new_g2) &
        (ratio1_orig_g3 == ratio1_new_g3))
        # (ratio2_orig_g1 == ratio1_new_g1) &
        # (ratio2_orig_g2 == ratio2_new_g2))

In [13]:
print(bc_source.shape, bc_target.shape)

(212, 10) (212, 10)


# DiCE

In [17]:
from sklearn.linear_model import LogisticRegression
from src.logistic_regression import PTLogisticRegression, PTNNSimple, FFNetwork, PTNN
from src.training import regular_training
import torch
import pandas as pd
from src.training import dro_training
from src.cf_transport import get_dice_transformed, get_closest_target
from src.distance import percent_flipped
from src.distance import group_percent_explained


adult_raw_data = adult_raw_data.astype(float)

adult_full = pd.concat([adult_source_data, adult_target_data])
maxabs = adult_raw_data.drop(columns=["income"]).max()

x = adult_full.drop(columns=["income"]) / maxabs
x = torch.from_numpy(x.to_numpy()).float()
y = adult_full['income']
y = torch.from_numpy(y.to_numpy()[:, np.newaxis]).float()

group_mat = np.concatenate([adult_source_feasible_groups, adult_target_feasible_groups], axis=0)
group_mat = np.concatenate([group_mat, np.ones((group_mat.shape[0], 1))], axis=1)
group_mat = torch.from_numpy(group_mat).double()

In [15]:
# Training an embedding model
emb = PTNN(35, 1)
regular_training(emb, x, y, group_mat, 100, 0.05, 1e-3)

tensor(0.5000)


0 0.6926242709159851
0 0.6926328539848328
0 0.6933477520942688
0 0.694329023361206
0 0.6947747468948364
0 0.6925832629203796
0 0.6936944127082825
0 0.693881630897522
0 0.693408727645874
0 0.6942548751831055
0 0.6929177045822144
0 0.6923307180404663
0 0.693730890750885
0 0.6910185217857361
0 0.6914766430854797
0 0.6909810900688171
1 0.6919402480125427
1 0.6892383694648743
1 0.6899023652076721
1 0.6887461543083191
1 0.6915842890739441
1 0.689237117767334
1 0.6903971433639526
1 0.6878342628479004
1 0.6887599229812622
1 0.6879183650016785
1 0.6885294318199158
1 0.6881740093231201
1 0.6879870295524597
1 0.6880428791046143
1 0.687933623790741
1 0.6873198747634888
2 0.6866477727890015
2 0.6874951720237732
2 0.6855157613754272
2 0.6853117942810059
2 0.6853160262107849
2 0.6851968169212341
2 0.6826499104499817
2 0.6839130520820618
2 0.6800603270530701
2 0.6800428628921509
2 0.6801342964172363
2 0.6819314360618591
2 0.6774135231971741
2 0.6763691306114197
2 0.6792535781860352
2 0.673958659172058

24 0.4784708321094513
24 0.4466214179992676
24 0.4330257773399353
24 0.4625089764595032
24 0.4485210180282593
24 0.45464974641799927
24 0.4844401478767395
24 0.45853570103645325
24 0.4655599594116211
24 0.47523343563079834
24 0.45841553807258606
24 0.48848143219947815
24 0.46214771270751953
24 0.5061083436012268
25 0.47033482789993286
25 0.45390716195106506
25 0.44000887870788574
25 0.49660786986351013
25 0.44564521312713623
25 0.5020043253898621
25 0.4936472475528717
25 0.45893824100494385
25 0.45849141478538513
25 0.4567609429359436
25 0.43656209111213684
25 0.48693886399269104
25 0.4518248438835144
25 0.46972671151161194
25 0.4626576900482178
25 0.4710490107536316
26 0.4483577013015747
26 0.46782979369163513
26 0.4530050456523895
26 0.4392211437225342
26 0.4437832534313202
26 0.4912594258785248
26 0.4574601352214813
26 0.4834747314453125
26 0.4688262641429901
26 0.46956202387809753
26 0.4822002053260803
26 0.4351501166820526
26 0.45878365635871887
26 0.4523969292640686
26 0.44774523

47 0.42321619391441345
47 0.46017178893089294
47 0.4482792019844055
47 0.43956056237220764
47 0.4341638386249542
47 0.42357659339904785
47 0.42881572246551514
48 0.43555307388305664
48 0.43540528416633606
48 0.44237184524536133
48 0.44249895215034485
48 0.4572179317474365
48 0.41921889781951904
48 0.4387385845184326
48 0.42580366134643555
48 0.4507230222225189
48 0.43172022700309753
48 0.45650777220726013
48 0.4635162055492401
48 0.4284187853336334
48 0.481845498085022
48 0.44799676537513733
48 0.47780415415763855
49 0.46555766463279724
49 0.44451814889907837
49 0.4550553560256958
49 0.42608514428138733
49 0.4230995774269104
49 0.4251675307750702
49 0.45956602692604065
49 0.4542950689792633
49 0.45079341530799866
49 0.4731569290161133
49 0.4510927200317383
49 0.4528799057006836
49 0.4431644380092621
49 0.4344154894351959
49 0.44455021619796753
49 0.43227827548980713
50 0.4311920404434204
50 0.4653221666812897
50 0.42326217889785767
50 0.43900465965270996
50 0.4415828585624695
50 0.4159

70 0.47351622581481934
70 0.4284444749355316
70 0.4340657591819763
70 0.4579101800918579
71 0.45022761821746826
71 0.43459877371788025
71 0.43630146980285645
71 0.43251144886016846
71 0.42380449175834656
71 0.3953222334384918
71 0.43220168352127075
71 0.4275044798851013
71 0.5003467202186584
71 0.45389264822006226
71 0.4309956133365631
71 0.470801442861557
71 0.4642287790775299
71 0.4342552125453949
71 0.44719791412353516
71 0.4503612816333771
72 0.40219125151634216
72 0.4170380234718323
72 0.41967612504959106
72 0.4557173550128937
72 0.4570143520832062
72 0.4859916567802429
72 0.44145822525024414
72 0.47385305166244507
72 0.4450330436229706
72 0.4268587529659271
72 0.423665851354599
72 0.45091909170150757
72 0.42480581998825073
72 0.4613199234008789
72 0.44507691264152527
72 0.40969690680503845
73 0.42537012696266174
73 0.4472813904285431
73 0.3982205390930176
73 0.44281837344169617
73 0.4375341534614563
73 0.44803038239479065
73 0.4496937394142151
73 0.4251616299152374
73 0.456657201

93 0.4044739007949829
93 0.4468806982040405
94 0.414754182100296
94 0.46699488162994385
94 0.4015953838825226
94 0.42489251494407654
94 0.44762715697288513
94 0.4040597081184387
94 0.41961097717285156
94 0.4257923662662506
94 0.44041964411735535
94 0.44268539547920227
94 0.4238283038139343
94 0.4277435839176178
94 0.4308014214038849
94 0.4719066917896271
94 0.4406656324863434
94 0.41925737261772156
95 0.4261835813522339
95 0.4699735939502716
95 0.43431612849235535
95 0.4137853980064392
95 0.45705994963645935
95 0.42267701029777527
95 0.4233132600784302
95 0.439725786447525
95 0.4164620041847229
95 0.42722970247268677
95 0.4474325478076935
95 0.438188374042511
95 0.451793909072876
95 0.43383657932281494
95 0.4676140248775482
95 0.42144134640693665
96 0.43928930163383484
96 0.4485114812850952
96 0.4329446852207184
96 0.43857064843177795
96 0.4441593885421753
96 0.4192776083946228
96 0.4406607151031494
96 0.448902428150177
96 0.43759676814079285
96 0.4379745423793793
96 0.42861208319664
9

torch.Size([15682, 1]) torch.Size([15682, 1])
Accuracy: tensor(0.8098)
tensor([0.7886, 0.8668, 0.8098], dtype=torch.float64)


99 0.44498446583747864
99 0.4324471354484558


In [27]:
reg_total = []
reg_worst = []
reg_feas = []
reg_pf = []
g_total = []
g_worst = []
g_feas = []
g_pf = []
for i in range(3):
    np.random.seed(i)
    lr = FFNetwork(35)
    regular_training(lr, x, y, group_mat, 100, 0.05, 1e-4)

    lr_group = FFNetwork(35)
    dro_training(lr_group, x, y, group_mat, 100, 0.05, 1e-4)

    sample_source = np.random.permutation(adult_source.shape[0])[:500]

    x_s = get_dice_transformed(
        lr,
        adult_full / adult_full.max(),
        adult_source_data.drop(columns=['income']).iloc[sample_source] / adult_full.drop(columns=['income']).max(),
        "income",
        0)
    x_s_group = get_dice_transformed(
        lr_group,
        adult_full / adult_full.max(),
        adult_source_data.drop(columns=['income']).iloc[sample_source] / adult_full.drop(columns=['income']).max(),
        "income",
        0)

    reg_probs = lr(torch.from_numpy(adult_target).float())
    dro_probs = lr_group(torch.from_numpy(adult_target).float())
    probs = (reg_probs + dro_probs) / 2
    probs = torch.abs(torch.max(probs, dim=1)[0] - 0.5)
    closest_target_to_boundary = torch.argsort(probs)[-500:]

    norm_const = adult_full.drop(columns=['income']).max().to_numpy().astype(float)
    x_s_norm = (x_s.to_numpy().astype(float))# * norm_const) #.round(0)

    total, worst = group_percent_explained(adult_source[sample_source],
                            x_s_norm,
                            adult_target[closest_target_to_boundary],
                            adult_source_feasible_groups[sample_source],
                            adult_target_feasible_groups[closest_target_to_boundary],
                            adult_feasible_names)
    x_s_group_norm = (x_s_group.to_numpy().astype(float))# * norm_const) #.round(0)

    group_total, group_worst = group_percent_explained(adult_source[sample_source],
                            x_s_group_norm,
                            adult_target[closest_target_to_boundary],
                            adult_source_feasible_groups[sample_source],
                            adult_target_feasible_groups[closest_target_to_boundary],
                            adult_feasible_names)
    reg_total.append(total)
    reg_worst.append(worst)
    g_total.append(group_total)
    g_worst.append(group_worst)
    
    feasible = is_adult_feasible_change(adult_source[sample_source], x_s_norm)
    reg_feas.append(np.sum(feasible) / feasible.shape[0])

    group_feasible = is_adult_feasible_change(adult_source[sample_source], x_s_group_norm)
    g_feas.append(np.sum(group_feasible) / group_feasible.shape[0])
    
    reg_pf.append(percent_flipped(emb, adult_source[sample_source], x_s_norm, adult_source_feasible_groups[sample_source]))
    g_pf.append(percent_flipped(emb, adult_source[sample_source], x_s_group_norm, adult_source_feasible_groups[sample_source]))

tensor(0.5000)


0 0.7114306092262268
0 0.7031329870223999
0 0.7086279392242432
0 0.7031160593032837
0 0.6941739320755005
0 0.6946257948875427
0 0.6905711889266968
0 0.6885738968849182
0 0.6871935725212097
0 0.6850898861885071
0 0.6817871332168579
0 0.6833844184875488
0 0.6806954145431519
0 0.6787294745445251
0 0.6781213879585266
0 0.6718128323554993
1 0.676376223564148
1 0.6716572046279907
1 0.6675111055374146
1 0.6697366833686829
1 0.659596860408783
1 0.6573435664176941
1 0.6522337794303894
1 0.6511258482933044
1 0.6499917507171631
1 0.646492600440979
1 0.6391904354095459
1 0.6364501118659973
1 0.624161958694458
1 0.628748893737793
1 0.6257641911506653
1 0.6235671043395996
2 0.6086695194244385
2 0.6088129878044128
2 0.597714364528656
2 0.5987696647644043
2 0.579334020614624
2 0.5772580504417419
2 0.5664634704589844
2 0.5760918259620667
2 0.5487131476402283
2 0.5526944398880005
2 0.5410914421081543
2 0.5480542182922363
2 0.5411916971206665
2 0.5350086092948914
2 0.5462576150894165
2 0.5184813737869263

24 0.4286118447780609
24 0.4507870376110077
25 0.4299345314502716
25 0.4277571737766266
25 0.4325144290924072
25 0.42291316390037537
25 0.41467782855033875
25 0.442985862493515
25 0.4343278408050537
25 0.4154350161552429
25 0.4185355305671692
25 0.4417465031147003
25 0.435985803604126
25 0.41944369673728943
25 0.45072582364082336
25 0.41605430841445923
25 0.4456917345523834
25 0.4455547034740448
26 0.4371449649333954
26 0.4152810871601105
26 0.42654672265052795
26 0.4120372533798218
26 0.4089462459087372
26 0.4283783733844757
26 0.4781813323497772
26 0.4055693447589874
26 0.4407082200050354
26 0.43516623973846436
26 0.4310017228126526
26 0.44302985072135925
26 0.45054733753204346
26 0.42191511392593384
26 0.4407804608345032
26 0.4043155014514923
27 0.43453115224838257
27 0.430502712726593
27 0.41472962498664856
27 0.40479788184165955
27 0.4363512694835663
27 0.39936158061027527
27 0.41914382576942444
27 0.43224993348121643
27 0.44811129570007324
27 0.4286282956600189
27 0.4592716097831

48 0.4091736674308777
48 0.4195009171962738
48 0.43335801362991333
48 0.4425215721130371
48 0.44470396637916565
48 0.40665656328201294
48 0.41722944378852844
48 0.45244282484054565
48 0.42416512966156006
48 0.413158655166626
48 0.42297878861427307
48 0.4325167238712311
49 0.43599921464920044
49 0.4066063463687897
49 0.4248078167438507
49 0.41909417510032654
49 0.4335339665412903
49 0.427642822265625
49 0.4556989371776581
49 0.43059372901916504
49 0.43194666504859924
49 0.4228290617465973
49 0.41210195422172546
49 0.4141235053539276
49 0.4255663752555847
49 0.41894984245300293
49 0.43784645199775696
49 0.4457536041736603
50 0.4290350675582886
50 0.45541542768478394
50 0.41193318367004395
50 0.4307191073894501
50 0.40220585465431213
50 0.42883771657943726
50 0.4527156352996826
50 0.42743033170700073
50 0.4465067982673645
50 0.39585983753204346
50 0.4397684633731842
50 0.43109607696533203
50 0.40455570816993713
50 0.421597421169281
50 0.4248203933238983
50 0.4278705418109894
51 0.42646893

71 0.43180373311042786
71 0.4372890591621399
72 0.44315218925476074
72 0.4263885021209717
72 0.4167623817920685
72 0.4018983244895935
72 0.4182528555393219
72 0.42797064781188965
72 0.4065208435058594
72 0.4145181179046631
72 0.43742749094963074
72 0.4350612461566925
72 0.3958861529827118
72 0.44842588901519775
72 0.4324275553226471
72 0.4391232430934906
72 0.42911621928215027
72 0.42368581891059875
73 0.4369142949581146
73 0.43474799394607544
73 0.43299755454063416
73 0.42700520157814026
73 0.41592398285865784
73 0.39450135827064514
73 0.45108693838119507
73 0.43413257598876953
73 0.43076395988464355
73 0.42304933071136475
73 0.42073026299476624
73 0.42507681250572205
73 0.43884775042533875
73 0.42307186126708984
73 0.4006516635417938
73 0.39304813742637634
74 0.41939377784729004
74 0.40266433358192444
74 0.41894081234931946
74 0.43754932284355164
74 0.4273762106895447
74 0.4370472729206085
74 0.40961670875549316
74 0.401452898979187
74 0.42928072810173035
74 0.41298800706863403
74 0.

...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 499/500 [02:21<00:00,  3.40it/s]

Diverse Counterfactuals found! total time taken: 00 min 00 sec


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [02:21<00:00,  3.54it/s]

Diverse Counterfactuals found! total time taken: 00 min 00 sec


Total Percent Explained: 2.1462934932604494
male Percent Explained: 3.728526840668353
female Percent Explained: 2.78392703528667
Worst group PE: 2.78392703528667
Total Percent Explained: 22.060869193437835
male Percent Explained: 27.107743881646634
female Percent Explained: 17.296214533756
Worst group PE: 17.296214533756
87.8 [91.08433735 71.76470588]
85.8 [89.15662651 69.41176471]


In [28]:
print(np.mean(reg_total), np.std(reg_total))
print(np.mean(reg_worst), np.std(reg_worst))
print(np.mean(reg_feas), np.std(reg_feas))
print(np.mean(reg_pf), np.std(reg_pf))
print(np.mean(g_total), np.std(g_total))
print(np.mean(g_worst), np.std(g_worst))
print(np.mean(g_feas), np.std(g_feas))
print(np.mean(g_pf), np.std(g_pf))

2.2517935038790724 0.2925857743010315
2.4597296330130427 0.23517888511226845
1.0 0.0
0.8846666666666666 0.008055363982396388
26.02077745482336 3.000759685914799
21.690160878588276 4.768640877282272
1.0 0.0
0.8793333333333333 0.01517307556898807


## Breast Cancer

In [43]:
from src.logistic_regression import PTLogisticRegression, PTNN, PTNNSimple, FFNetwork
from src.training import regular_training, dro_training
from src.distance import group_percent_explained
from src.cf_transport import get_dice_transformed, get_closest_target
import torch
from src.training import dro_training
from src.distance import percent_flipped

bc_source_df['diagnosis'] = 1
bc_target_df['diagnosis'] = 0

bc_full = pd.concat([bc_source_df, bc_target_df])
x = torch.from_numpy((bc_full / bc_full.max()).drop(columns=['diagnosis']).to_numpy()).float()
y = torch.from_numpy(bc_full['diagnosis'].to_numpy()[:, np.newaxis]).flatten().float()[:, None]
group_mat = np.concatenate([bc_source_feasible_groups, bc_target_feasible_groups], axis=0)
group_mat = np.concatenate([group_mat, np.ones((group_mat.shape[0], 1))], axis=1)
group_mat = torch.from_numpy(group_mat).double()

In [44]:
# Training an embedding model
emb = PTNN(10, 1)
regular_training(emb, x, y, group_mat, 500, 0.1, 5e-5)

tensor(0.5000)


0 0.6935291290283203
1 0.6935100555419922
2 0.6935723423957825
3 0.694476306438446
4 0.6935303211212158
5 0.6921923756599426
6 0.6932213306427002
7 0.6897259950637817
8 0.6914723515510559
9 0.6917658448219299
10 0.6902071237564087
11 0.689195454120636
12 0.6913849711418152
13 0.6902029514312744
14 0.6902322769165039
15 0.6866704225540161
16 0.6890113949775696
17 0.6874831914901733
18 0.6845861077308655
19 0.6852178573608398
20 0.6807857155799866
21 0.6834864616394043
22 0.6796413660049438
23 0.679563581943512
24 0.6782081723213196
25 0.6734475493431091
26 0.6772372722625732
27 0.6670031547546387
28 0.6699537038803101
29 0.6551082134246826
30 0.6548970937728882
31 0.652949869632721
32 0.6455049514770508
33 0.6284974813461304
34 0.6231667995452881
35 0.6127908229827881
36 0.6033947467803955
37 0.586425244808197
38 0.5921971201896667
39 0.5457115173339844
40 0.5475689768791199
41 0.5416544675827026
42 0.5097655653953552
43 0.48308494687080383
44 0.4610907733440399
45 0.455617755651474
46 

371 0.17693589627742767
372 0.16604852676391602
373 0.17273356020450592
374 0.20687884092330933
375 0.16369223594665527
376 0.18700331449508667
377 0.1533498615026474
378 0.17790193855762482
379 0.1720605492591858
380 0.1686553657054901
381 0.1578923910856247
382 0.17486213147640228
383 0.186307892203331
384 0.17301088571548462
385 0.16976384818553925
386 0.1711169183254242
387 0.19121047854423523
388 0.1737513244152069
389 0.1737949252128601
390 0.16741102933883667
391 0.14419756829738617
392 0.17428766191005707
393 0.19367659091949463
394 0.15758737921714783
395 0.1774093359708786
396 0.19799745082855225
397 0.16693980991840363
398 0.20897912979125977
399 0.17211495339870453
400 0.16885901987552643
401 0.1696292608976364
402 0.17600639164447784
403 0.18965288996696472
404 0.1910812258720398
405 0.1727888435125351
406 0.1750854253768921
407 0.17660969495773315
408 0.18599684536457062
409 0.1797793060541153
410 0.18236462771892548
411 0.16744153201580048
412 0.1853484958410263
413 0.16

torch.Size([424, 1]) torch.Size([424, 1])
Accuracy: tensor(0.9410)
tensor([0.9245, 0.9481, 0.9434, 0.9410], dtype=torch.float64)


482 0.17212560772895813
483 0.15321555733680725
484 0.16770637035369873
485 0.1691572666168213
486 0.16801749169826508
487 0.16450630128383636
488 0.1567089706659317
489 0.1554858684539795
490 0.15187513828277588
491 0.1820632368326187
492 0.1815672665834427
493 0.19059966504573822
494 0.1701812893152237
495 0.1726790815591812
496 0.16537873446941376
497 0.157253235578537
498 0.1810477375984192
499 0.1719372272491455


In [45]:
reg_total = []
reg_worst = []
reg_feas = []
reg_pf = []
group_total = []
group_worst = []
group_feas = []
group_pf = []
for i in range(3):
    np.random.seed(i)
    lr = FFNetwork(10)
    regular_training(lr, x, y, group_mat, 500, 0.2, 1e-4)

    lr_group = FFNetwork(10)
    dro_training(lr_group, x, y, group_mat, 500, 0.2, 1e-4)

    np.random.seed(0)
    sample_source = np.random.permutation(bc_source_df.shape[0])
    # sample_target = np.random.permutation(adult_target.shape[0])[:100]
    # lr_p = torch.nn.Sequential(lr, torch.nn.Softmax(dim=0))
    x_s = get_dice_transformed(
        lr,
        (bc_full / bc_full.max()),
        (bc_source_df / bc_full.max()).drop(columns=['diagnosis']).iloc[sample_source],
        "diagnosis",
        0)

    x_s_group = get_dice_transformed(
        lr_group,
        (bc_full / bc_full.max()),
        (bc_source_df / bc_full.max()).drop(columns=['diagnosis']).iloc[sample_source],
        "diagnosis",
        0)

    reg_probs = lr(torch.from_numpy(bc_target).float())
    dro_probs = lr_group(torch.from_numpy(bc_target).float())
    probs = (reg_probs + dro_probs) / 2
    probs = torch.abs(torch.max(probs, dim=1)[0] - 0.5)
    closest_target_to_boundary = torch.argsort(probs)[-500:]

    x_s_norm = x_s.to_numpy().astype(float)
    total, worst = group_percent_explained(
        bc_source[sample_source],
        x_s_norm,
        bc_target,
        bc_source_feasible_groups[sample_source],
        bc_target_feasible_groups,
        bc_feasible_names)

    x_s_group_norm = x_s_group.to_numpy().astype(float)
    g_total, g_worst = group_percent_explained(
        bc_source[sample_source],
        x_s_group_norm,
        bc_target,
        bc_source_feasible_groups[sample_source],
        bc_target_feasible_groups,
        bc_feasible_names)
    
    reg_total.append(total)
    reg_worst.append(worst)
    group_total.append(g_total)
    group_worst.append(g_worst)    

    feasible = is_bc_feasible_change(bc_source[sample_source], x_s_norm)
    reg_feas.append(np.sum(feasible) / feasible.shape[0])

    feasible = is_bc_feasible_change(bc_source[sample_source], x_s_group_norm)
    group_feas.append(np.sum(feasible) / feasible.shape[0])

    reg_pf.append(percent_flipped(emb, bc_source[sample_source], x_s_norm, bc_source_feasible_groups[sample_source]))
    group_pf.append(percent_flipped(emb, bc_source[sample_source], x_s_group_norm, bc_source_feasible_groups[sample_source]))

tensor(0.5000)


0 0.6924058794975281
1 0.690669059753418
2 0.6873360276222229
3 0.6822998523712158
4 0.6764907240867615
5 0.6713460683822632
6 0.6670529246330261
7 0.6627237796783447
8 0.6572650074958801
9 0.649930477142334
10 0.640516459941864
11 0.6290421485900879
12 0.6153494119644165
13 0.5992163419723511
14 0.5804017782211304
15 0.5588755011558533
16 0.5347152352333069
17 0.5081814527511597
18 0.4796735644340515
19 0.44985002279281616
20 0.4196636378765106
21 0.39014896750450134
22 0.36213943362236023
23 0.33631080389022827
24 0.3132781982421875
25 0.2933172881603241
26 0.27634021639823914
27 0.2621515691280365
28 0.2506076693534851
29 0.2413429468870163
30 0.23392605781555176
31 0.22807110846042633
32 0.22351263463497162
33 0.2199247181415558
34 0.21706993877887726
35 0.214813232421875
36 0.21299107372760773
37 0.21144716441631317
38 0.21011093258857727
39 0.20893311500549316
40 0.20783360302448273
41 0.20676517486572266
42 0.20572848618030548
43 0.20470629632472992
44 0.20366622507572174
45 0.2

372 0.14561228454113007
373 0.1455823928117752
374 0.1455518752336502
375 0.14552666246891022
376 0.14549316465854645
377 0.14546965062618256
378 0.14543481171131134
379 0.14541105926036835
380 0.145376056432724
381 0.1453484445810318
382 0.14531925320625305
383 0.1452891081571579
384 0.1452600657939911
385 0.14523306488990784
386 0.14520364999771118
387 0.14517392218112946
388 0.14514577388763428
389 0.14511755108833313
390 0.14509502053260803
391 0.14506796002388
392 0.14503072202205658
393 0.14501215517520905
394 0.1449722945690155
395 0.1449483186006546
396 0.14491620659828186
397 0.14489038288593292
398 0.1448613554239273
399 0.144837886095047
400 0.1448073387145996
401 0.14478208124637604
402 0.14474530518054962
403 0.14473657310009003
404 0.14468882977962494
405 0.14467349648475647
406 0.14464712142944336
407 0.14461293816566467
408 0.1445860117673874
409 0.1445525884628296
410 0.14452652633190155
411 0.1444990634918213
412 0.14447052776813507
413 0.144443541765213
414 0.1444125

torch.Size([424, 1]) torch.Size([424, 1])
Accuracy: tensor(0.9340)
tensor([0.9340, 0.9245, 0.9528, 0.9340], dtype=torch.float64)


46 0.19539650905639572
47 0.19796365023441864
48 0.14108533745314547
49 0.21237325711113603
50 0.20565307767843372
51 0.18898747535897015
52 0.22001508768211875
53 0.15184702369063635
54 0.20305274192435596
55 0.16527489202171392
56 0.19211483697714235
57 0.20458733742775154
58 0.18613111071197425
59 0.2033229478438764
60 0.18722371684762792
61 0.18640091752419685
62 0.18292292644849645
63 0.18307145836525165
64 0.17401610609020773
65 0.17243964138761894
66 0.16217131346340097
67 0.23170734014492905
68 0.16559489240788106
69 0.20135608496598367
70 0.17628081698974707
71 0.18969144706942945
72 0.17368409626126813
73 0.17999113339677608
74 0.16708014574582172
75 0.15892369227675582
76 0.19727062958492764
77 0.17217976158025916
78 0.15131904239476443
79 0.19732700863240465
80 0.24115333706845668
81 0.19331382215657553
82 0.20743056392740147
83 0.1353301939857191
84 0.1970297562047274
85 0.14217834096694665
86 0.17231923226493703
87 0.13958591163697898
88 0.16816986860827687
89 0.192396012

...

Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 211/212 [00:31<00:00,  6.98it/s]

Diverse Counterfactuals found! total time taken: 00 min 00 sec
Diverse Counterfactuals found! total time taken: 00 min 00 sec


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 212/212 [00:31<00:00,  6.75it/s]


Diverse Counterfactuals found! total time taken: 00 min 00 sec
Total Percent Explained: 30.599917075260908
radius**2 / area above third quartile Percent Explained: 25.619178197488928
radius**2 / area between first and third quartile Percent Explained: 31.15678051362705
radius**2 / area below first quartile Percent Explained: 34.177814231511704
Worst group PE: 25.619178197488928
Total Percent Explained: 36.40598474485402
radius**2 / area above third quartile Percent Explained: 33.849563180353556
radius**2 / area between first and third quartile Percent Explained: 35.37673367076618
radius**2 / area below first quartile Percent Explained: 37.78094926250744
Worst group PE: 33.849563180353556
88.67924528301887 [84.90566038 89.62264151 90.56603774]
91.50943396226415 [83.01886792 94.33962264 94.33962264]


In [46]:
print(np.mean(reg_total), np.std(reg_total))
print(np.mean(reg_worst), np.std(reg_worst))
print(np.mean(reg_feas), np.std(reg_feas))
print(np.mean(reg_pf), np.std(reg_pf))
print(np.mean(group_total), np.std(group_total))
print(np.mean(group_worst), np.std(group_worst))
print(np.mean(group_feas), np.std(group_feas))
print(np.mean(group_pf), np.std(group_pf))

29.6029648108092 2.4297232344230597
25.16027398547476 0.9990561103007621
0.25943396226415094 0.0
0.8946540880503145 0.0058831090987011446
38.210785614608746 1.5845684864892966
33.47639684759421 0.40288707842493715
0.27201257861635225 0.014581161156439782
0.919811320754717 0.003851398966640216


# Worst Group k-Cluster Shift

## Adult

In [27]:
scaler = adult_scaler

source = scaler.inverse_transform(adult_source)
target = scaler.inverse_transform(adult_target)
source_feasible_groups = adult_source_feasible_groups
target_feasible_groups = adult_target_feasible_groups
feasible_names = adult_feasible_names
is_feasible_change = is_adult_feasible_change

In [28]:
from sklearn.model_selection import train_test_split
from src.training import eval_acc
from src.logistic_regression import PTLogisticRegression, PTNN, PTNNSimple, FFNetwork
import torch
from src.training import regular_training, dro_training
from src.distance import group_percent_explained
from src.cf_transport import get_dice_transformed, get_closest_target
import torch
from src.training import dro_training
from src.distance import percent_flipped

# Training an embedding model
adult_emb = PTNN(35, 1)
no_group = torch.ones((source.shape[0] + target.shape[0], 1))
X = torch.from_numpy(np.concatenate([source, target], axis=0)).float()
y = torch.from_numpy(np.concatenate([np.ones((source.shape[0], 1)), np.zeros((target.shape[0], 1))], axis=0)).float()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
regular_training(adult_emb,
                 X_train,
                 y_train,
                 torch.ones_like(y_train), 100, 0.5, 5e-5)
eval_acc(adult_emb, X_test, y_test, torch.ones_like(y_test))

tensor(0.5019)


0 0.7839217782020569
0 0.8099374771118164
0 0.6937723159790039
0 0.6935533285140991
0 0.6962622404098511
0 0.6993047595024109
0 0.6937870979309082
0 0.6967512369155884
0 0.6950002312660217
0 0.6919889450073242
0 0.6932128071784973
0 0.6903347373008728
0 0.6961324214935303
1 0.692073404788971
1 0.6841870546340942
1 0.7093493342399597
1 0.7142277956008911
1 0.6933383941650391
1 0.7085009813308716
1 0.6988677382469177
1 0.6858042478561401
1 0.6887180209159851
1 0.6917253732681274
1 0.6956753134727478
1 0.6929535865783691
1 0.6835444569587708
2 0.6987771391868591
2 0.689883828163147
2 0.692937433719635
2 0.6905674338340759
2 0.6888784766197205
2 0.6868093013763428
2 0.6900484561920166
2 0.6924849152565002
2 0.6899424195289612
2 0.6898038387298584
2 0.685086190700531
2 0.6917210817337036
2 0.6850364804267883
3 0.6900345087051392
3 0.6918188333511353
3 0.6905748844146729
3 0.6892569661140442
3 0.6859701871871948
3 0.6880278587341309
3 0.6847456097602844
3 0.6944012641906738
3 0.6871067285537

29 0.6941213607788086
29 0.6949788928031921
29 0.6927932500839233
29 0.6931361556053162
29 0.6933758854866028
29 0.6927772164344788
29 0.6945381164550781
30 0.6930854320526123
30 0.6935136318206787
30 0.6929007768630981
30 0.6931005716323853
30 0.6931620240211487
30 0.6932055950164795
30 0.6931556463241577
30 0.6931958198547363
30 0.693073034286499
30 0.6931034922599792
30 0.6939631700515747
30 0.6919410824775696
30 0.6938663721084595
31 0.6926711797714233
31 0.6941465139389038
31 0.6932502388954163
31 0.6930333375930786
31 0.6930002570152283
31 0.6933679580688477
31 0.6932038068771362
31 0.693394124507904
31 0.6931316256523132
31 0.6932497024536133
31 0.6935714483261108
31 0.6931115984916687
31 0.6932003498077393
32 0.6931573748588562
32 0.6928420662879944
32 0.6932811737060547
32 0.6938979625701904
32 0.6940594911575317
32 0.6925468444824219
32 0.6935033202171326
32 0.6938360929489136
32 0.6930679082870483
32 0.6931269764900208
32 0.6930737495422363
32 0.6934100389480591
32 0.6923757

58 0.693141758441925
58 0.6933373212814331
58 0.693130612373352
58 0.6931175589561462
58 0.6932903528213501
58 0.6935504078865051
58 0.6926984190940857
59 0.6938482522964478
59 0.6932507753372192
59 0.6931260228157043
59 0.6931482553482056
59 0.6932973265647888
59 0.6930414438247681
59 0.6933021545410156
59 0.6931734681129456
59 0.6931090354919434
59 0.6931516528129578
59 0.6931034922599792
59 0.6930137276649475
59 0.6940953731536865
60 0.69297856092453
60 0.6930010914802551
60 0.6931670308113098
60 0.6926162242889404
60 0.6927435398101807
60 0.694693386554718
60 0.6943516731262207
60 0.693559467792511
60 0.6932774782180786
60 0.693134069442749
60 0.6933758854866028
60 0.6925575137138367
60 0.694317102432251
61 0.6946163773536682
61 0.6935235857963562
61 0.6928830146789551
61 0.6930400729179382
61 0.6931729912757874
61 0.6929413080215454
61 0.6938692927360535
61 0.692621111869812
61 0.6942428946495056
61 0.6928274631500244
61 0.6930908560752869
61 0.693477988243103
61 0.692957282066345

87 0.6930670142173767
87 0.6929155588150024
87 0.6927739381790161
87 0.6942867636680603
87 0.6933510899543762
87 0.6931570768356323
88 0.6929901242256165
88 0.69337397813797
88 0.6931421160697937
88 0.6931440830230713
88 0.6933580636978149
88 0.6931173205375671
88 0.6932447552680969
88 0.6931954622268677
88 0.6931551694869995
88 0.6931490898132324
88 0.6931600570678711
88 0.6930727958679199
88 0.69334876537323
89 0.693310558795929
89 0.6932892203330994
89 0.6931464076042175
89 0.6931460499763489
89 0.6930513381958008
89 0.6931344270706177
89 0.6928428411483765
89 0.6929089426994324
89 0.6945832967758179
89 0.6942744851112366
89 0.6932798624038696
89 0.6931636929512024
89 0.6928709149360657
90 0.6933092474937439
90 0.6943476796150208
90 0.6936017274856567
90 0.6928874850273132
90 0.693101167678833
90 0.6930274963378906
90 0.6932905316352844
90 0.693278968334198
90 0.6930232644081116
90 0.6943705081939697
90 0.6929517984390259
90 0.6940429210662842
90 0.6966166496276855
91 0.692779123783

torch.Size([12545, 1]) torch.Size([12545, 1])
Accuracy: tensor(0.5019)
tensor([0.5019])
Accuracy: tensor(0.4925)
tensor([0.4925])


99 0.6931393146514893
99 0.6933557391166687
99 0.693167507648468
99 0.6933062076568604
99 0.69305819272995
99 0.6931474804878235
99 0.6932902336120605
99 0.693174421787262


In [30]:
from src.optimal_transport import group_mean_shift_transport, group_kmeans_shift_transport, transform_samples_kmeans
from src.distance import group_percent_explained
import pickle

np.random.seed(0)
sample_size = 1000
subset = np.random.permutation(source.shape[0])[:sample_size]
no_group_s = np.ones((sample_size, 1))
no_group_t = np.ones((sample_size, 1))

reg_total = []
reg_worst = []
reg_feas = []
reg_pf = []
reg_pf_worst = []
group_total = []
group_worst = []
group_feas = []
group_pf = []
group_pf_worst = []
for i in range(3):
    np.random.seed(i)

    x_s, centroids, shifts = group_kmeans_shift_transport(source[subset], target[subset], no_group_s, no_group_t, source.shape[1], 10, 1.0, 100)
    x_s_group, g_centroids, g_shifts = group_kmeans_shift_transport(source[subset], target[subset], source_feasible_groups[subset], target_feasible_groups[subset], source.shape[1], 10, 1.0, 100)

    total, worst = group_percent_explained(
        source[subset],
        (x_s).round(0),
        target[subset],
        source_feasible_groups[subset],
        target_feasible_groups[subset],
        feasible_names)

    feasible = is_feasible_change(
        source[subset],
        (x_s).round(0))
    reg_feas.append(np.sum(feasible) / feasible.shape[0])
    
    g_feasible = is_feasible_change(
        source[subset],
        (x_s_group).round(0))
    group_feas.append(np.sum(g_feasible) / g_feasible.shape[0])

    g_total, g_worst = group_percent_explained(
        source[subset],
        (x_s_group).round(0),
        target[subset],
        source_feasible_groups[subset],
        target_feasible_groups[subset],
        feasible_names)

    reg_total.append(total)
    reg_worst.append(worst)
    reg_feas.append((np.sum(feasible) / feasible.shape[0]))
    group_total.append(g_total)
    group_worst.append(g_worst)
    group_feas.append((np.sum(g_feasible) / g_feasible.shape[0]))
    
    pf, pf_worst = percent_flipped(adult_emb, source[subset], x_s, source_feasible_groups[subset])
    reg_pf.append(pf)
    reg_pf_worst.append(pf_worst)
    pf, pf_worst = percent_flipped(adult_emb, source[subset], x_s_group, source_feasible_groups[subset])
    group_pf.append(pf)
    group_pf_worst.append(pf_worst)

Samples in cluster 0: 104
Samples in cluster 1: 129
Samples in cluster 2: 90
Samples in cluster 3: 93
Samples in cluster 4: 103
Samples in cluster 5: 38
Samples in cluster 6: 98
Samples in cluster 7: 101
Samples in cluster 8: 119
Samples in cluster 9: 125
Iter 0 loss: 1.0
Change in x_s: 18.707326889038086
Iter 1 loss: 0.8014542996472898
Change in x_s: 9.326213836669922
Iter 2 loss: 0.6269858699067873
Change in x_s: 8.538884162902832
Iter 3 loss: 0.4900346013849994
Change in x_s: 6.636270523071289
Iter 4 loss: 0.4035144494529835
Change in x_s: 5.302089214324951
Iter 5 loss: 0.31871456790200814
Change in x_s: 5.207073211669922
Iter 6 loss: 0.23970698039394042
Change in x_s: 5.175662040710449
Iter 7 loss: 0.18413853300732294
Change in x_s: 4.52116060256958
Iter 8 loss: 0.15304884316491443
Change in x_s: 3.6548609733581543
Iter 9 loss: 0.12823271785387416
Change in x_s: 3.5070583820343018
Iter 10 loss: 0.10202127499538467
Change in x_s: 3.7398643493652344
Iter 11 loss: 0.08336185205999319


Iter 14 loss: 0.1021752378403992
Change in x_s: 2.479095697402954
Iter 15 loss: 0.10289965113803878
Change in x_s: 2.6824042797088623
Iter 16 loss: 0.10373123209521076
Change in x_s: 2.595390558242798
Iter 17 loss: 0.10872440497623695
Change in x_s: 2.1819136142730713
Iter 18 loss: 0.11508713254866909
Change in x_s: 1.8195098638534546
Iter 19 loss: 0.11773590514506063
Change in x_s: 1.8248573541641235
Iter 20 loss: 0.11568177825979531
Change in x_s: 1.954318642616272
Iter 21 loss: 0.11175232037750703
Change in x_s: 1.9066060781478882
Iter 22 loss: 0.10787202130644458
Change in x_s: 1.7173527479171753
Iter 23 loss: 0.10334542218057362
Change in x_s: 1.6057275533676147
Iter 24 loss: 0.09719275204182232
Change in x_s: 1.6332728862762451
Iter 25 loss: 0.09017499084468227
Change in x_s: 1.6368488073349
Iter 26 loss: 0.0837725079738099
Change in x_s: 1.5423407554626465
Iter 27 loss: 0.0783186990908208
Change in x_s: 1.43936026096344
Iter 28 loss: 0.0733603077993403
Change in x_s: 1.421420812

Iter 27 loss: 0.06773622382041247
Change in x_s: 1.4503787755966187
Iter 28 loss: 0.0630616010550064
Change in x_s: 1.4064635038375854
Iter 29 loss: 0.05864537502597218
Change in x_s: 1.4192168712615967
Iter 30 loss: 0.054909353265342915
Change in x_s: 1.3829621076583862
Iter 31 loss: 0.05234421818420262
Change in x_s: 1.280632495880127
Iter 32 loss: 0.05072435256183641
Change in x_s: 1.1932997703552246
Iter 33 loss: 0.04960921133920636
Change in x_s: 1.165613055229187
Iter 34 loss: 0.04893848091528172
Change in x_s: 1.1377617120742798
Iter 35 loss: 0.0488271808351821
Change in x_s: 1.0554721355438232
Iter 36 loss: 0.04913797048589006
Change in x_s: 0.9678758978843689
Iter 37 loss: 0.04944223786651358
Change in x_s: 0.9326605200767517
Iter 38 loss: 0.049568408837887386
Change in x_s: 0.9201331734657288
Iter 39 loss: 0.04963277134967477
Change in x_s: 0.8853036165237427
Iter 40 loss: 0.04973853169190443
Change in x_s: 0.8120083212852478
Iter 41 loss: 0.04975714770817639
Change in x_s: 0

Iter 45 loss: 0.0595001984161741
Change in x_s: 0.6737645268440247
Iter 46 loss: 0.05872962195820079
Change in x_s: 0.6397168040275574
Iter 47 loss: 0.0580297587246687
Change in x_s: 0.6171731948852539
Iter 48 loss: 0.05741494901427724
Change in x_s: 0.5947443842887878
Iter 49 loss: 0.056933020463548915
Change in x_s: 0.5692284107208252
Iter 50 loss: 0.05661015477664678
Change in x_s: 0.5338165760040283
Iter 51 loss: 0.056407444512462296
Change in x_s: 0.5003619194030762
Iter 52 loss: 0.056256622576516535
Change in x_s: 0.4912887513637543
Iter 53 loss: 0.05612988187226324
Change in x_s: 0.489268958568573
Iter 54 loss: 0.056086671764498885
Change in x_s: 0.4626416862010956
Iter 55 loss: 0.05614865886922257
Change in x_s: 0.4176763892173767
Iter 56 loss: 0.0562419657819106
Change in x_s: 0.3805226683616638
Iter 57 loss: 0.0562836161483466
Change in x_s: 0.36073219776153564
Iter 58 loss: 0.05627605268303003
Change in x_s: 0.34775975346565247
Iter 59 loss: 0.05622566430173799
Change in x_s

Iter 57 loss: 0.046636687962528595
Change in x_s: 0.38685914874076843
Iter 58 loss: 0.04661739594493975
Change in x_s: 0.38472625613212585
Iter 59 loss: 0.04656406969489022
Change in x_s: 0.36720895767211914
Iter 60 loss: 0.04650754319142593
Change in x_s: 0.348408043384552
Iter 61 loss: 0.04642761118983691
Change in x_s: 0.3439730405807495
Iter 62 loss: 0.04632781152683679
Change in x_s: 0.3367786407470703
Iter 63 loss: 0.04624499271393645
Change in x_s: 0.31652286648750305
Iter 64 loss: 0.04617875093899881
Change in x_s: 0.2919483482837677
Iter 65 loss: 0.046115520994460674
Change in x_s: 0.27963635325431824
Iter 66 loss: 0.04604684873813234
Change in x_s: 0.27274441719055176
Iter 67 loss: 0.0459837267199581
Change in x_s: 0.26680973172187805
Iter 68 loss: 0.04594163207953917
Change in x_s: 0.25932133197784424
Iter 69 loss: 0.04592837952665074
Change in x_s: 0.24424277245998383
Iter 70 loss: 0.04594101988445172
Change in x_s: 0.22557729482650757
Iter 71 loss: 0.04596039476007236
Chan

Iter 74 loss: 0.05926220985437335
Change in x_s: 0.17619533836841583
Iter 75 loss: 0.05923457112729144
Change in x_s: 0.17182785272598267
Iter 76 loss: 0.059199387335827
Change in x_s: 0.16746027767658234
Iter 77 loss: 0.05916303806682686
Change in x_s: 0.15848428010940552
Iter 78 loss: 0.059126332155270724
Change in x_s: 0.15273770689964294
Iter 79 loss: 0.05908944283974732
Change in x_s: 0.14840885996818542
Iter 80 loss: 0.05905757925499473
Change in x_s: 0.1405637264251709
Iter 81 loss: 0.05903113123005874
Change in x_s: 0.13476113975048065
Iter 82 loss: 0.05900978405065675
Change in x_s: 0.12747962772846222
Iter 83 loss: 0.05899479754667858
Change in x_s: 0.12072500586509705
Iter 84 loss: 0.058986379980615344
Change in x_s: 0.11433910578489304
Iter 85 loss: 0.058983504315075076
Change in x_s: 0.10974098742008209
Iter 86 loss: 0.05898429670437032
Change in x_s: 0.10533961653709412
Iter 87 loss: 0.05898695909005883
Change in x_s: 0.09918370842933655
Iter 88 loss: 0.058990718895132487

In [31]:
print(np.mean(reg_total), np.std(reg_total))
print(np.mean(reg_worst), np.std(reg_worst))
print(np.mean(reg_feas), np.std(reg_feas))
print(np.mean(reg_pf_worst), np.std(reg_pf_worst))
print()
print(np.mean(group_total), np.std(group_total))
print(np.mean(group_worst), np.std(group_worst))
print(np.mean(group_feas), np.std(group_feas))
print(np.mean(group_pf_worst), np.std(group_pf_worst))

93.6779263598461 0.02093722813394813
89.00126263891529 0.09920228213578013
1.0 0.0
0.0 0.0

93.47465968837787 0.010185678551645554
90.265488203679 0.007203975250338495
1.0 0.0
0.0 0.0


In [24]:
feasible = is_feasible_change(scaler.inverse_transform(source[subset]), scaler.inverse_transform(x_s))
print("Percent feasible shift:", (np.sum(feasible) / feasible.shape[0]) * 100)

Percent feasible shift: 77.5


In [212]:
print(scaler.inverse_transform(centroids).round(1))
print((shifts).round(1))
for shift in shifts:
    print([f"{adult_feature_names[i]}: {s:.2f}" for i, s in enumerate(shift.round(1)) if s != 0])

[[49.2 11.6  0.9  0.8 -0.   0.1  0.2  0.  -0.   0.4  0.2  0.1  0.   0.1
  -0.   0.8  0.   0.   0.   0.  -0.   0.   0.   0.   1.  -0.  -0.  -0.
  -0.   0.  -0.  -0.  -0.  -0.  -0. ]
 [43.  10.   0.9  0.9  0.  -0.   0.   0.   1.   0.   0.  -0.   0.   0.
  -0.   0.9  0.   0.   0.   0.   0.   0.1  0.   0.4 -0.   0.   0.   0.1
   0.   0.   0.   0.  -0.   0.1  0.1]
 [43.6 12.1  1.   0.9 -0.  -0.  -0.   0.   1.  -0.  -0.  -0.   0.   0.1
  -0.   0.9  0.   0.   0.   0.  -0.   0.   0.   0.   1.  -0.  -0.  -0.
  -0.   0.   0.  -0.  -0.  -0.   0. ]
 [44.1 13.6  0.9  0.8  0.   0.   0.2  0.   0.5  0.1  0.1  0.1  0.   0.1
   0.   0.8  0.   0.1  0.   0.   0.  -0.   0.   0.  -0.  -0.  -0.  -0.
  -0.   0.   1.  -0.  -0.   0.   0. ]
 [46.1 10.4  0.9  0.8  0.2  0.2  0.3  0.   0.   0.1  0.2  0.1  0.   0.1
  -0.   0.9  0.   0.   0.   0.   0.2  0.2  0.   0.2 -0.   0.1  0.   0.
   0.   0.   0.   0.2 -0.   0.   0.1]
 [43.8 11.6  0.9  0.9 -0.   0.  -0.   0.   0.7  0.2  0.1 -0.   0.   0.
  -0.   0.9 -0.   0.1  0

In [213]:
from src.optimal_transport import transform_samples_kmeans
import pickle

# subset = np.random.permutation(source.shape[0])[:1000]
x_s_group, centroids, shifts = group_kmeans_shift_transport(source[subset], target[subset], source_feasible_groups[subset], target_feasible_groups[subset], source.shape[1], 6, 2.0, 40)
# x_s = transform_samples_kmeans(source, centroids, shifts)
centroids_i = scaler.inverse_transform(centroids)
centroids_f = scaler.inverse_transform(centroids + shifts)
shifts = centroids_f - centroids_i
group_percent_explained(source[subset], x_s_group, target[subset], source_feasible_groups[subset], target_feasible_groups[subset], feasible_names)

with open("../data/adult/kmeans_x_s_group.pkl", "wb") as f:
    pickle.dump(x_s_group, f)
with open("../data/adult/kmeans_centroids_group.pkl", "wb") as f:
    pickle.dump(centroids_i, f)
with open("../data/adult/kmeans_shifts_group.pkl", "wb") as f:
    pickle.dump(shifts, f)
# x_s = pickle.load(open("../data/adult/kmeans_x_s_group.pkl", "rb"))

Samples in cluster 0: 100
Samples in cluster 1: 252
Samples in cluster 2: 143
Samples in cluster 3: 229
Samples in cluster 4: 156
Samples in cluster 5: 120
Iter 0 loss: 1.0
Iter 1 loss: 0.82276704097594
Iter 2 loss: 0.7265914688516294
Iter 3 loss: 0.6828512884894338
Iter 4 loss: 0.6611019627790691
Iter 5 loss: 0.6482235167366084
Iter 6 loss: 0.6399083151244269
Iter 7 loss: 0.634406989255485
Iter 8 loss: 0.6307580918898423
Iter 9 loss: 0.628352701688617
Iter 10 loss: 0.62665181751301
Iter 11 loss: 0.6253594952454828
Iter 12 loss: 0.6244017227259003
Iter 13 loss: 0.6236894918488892
Iter 14 loss: 0.6230898847925312
Iter 15 loss: 0.6225356394820571
Iter 16 loss: 0.6220726072240468
Iter 17 loss: 0.6217155148261648
Iter 18 loss: 0.6213853171212067
Iter 19 loss: 0.6210780990033761
Iter 20 loss: 0.6208103266749029
Iter 21 loss: 0.6205792105329517
Iter 22 loss: 0.6203879573855624
Iter 23 loss: 0.6202181342305062
Iter 24 loss: 0.6200685358250304
Iter 25 loss: 0.6199346135702344
Iter 26 loss: 0.6

In [214]:
feasible = is_feasible_change(scaler.inverse_transform(source[subset]), scaler.inverse_transform(x_s_group))
print("Percent feasible shift:", (np.sum(feasible) / feasible.shape[0]) * 100)

Percent feasible shift: 100.0


In [215]:
print(scaler.inverse_transform(centroids).round(1))
print((shifts).round(1))
for shift in shifts:
    print([f"{adult_feature_names[i]}: {s:.2f}" for i, s in enumerate(shift.round(1)) if s != 0])

[[49.2 11.6  0.9  0.8 -0.   0.1  0.2  0.  -0.   0.4  0.2  0.1  0.   0.1
  -0.   0.8  0.   0.   0.   0.  -0.   0.   0.   0.   1.  -0.  -0.  -0.
  -0.   0.  -0.  -0.  -0.  -0.   0. ]
 [43.  10.   0.9  0.9  0.  -0.   0.   0.   1.   0.   0.  -0.   0.   0.
  -0.   0.9  0.   0.   0.   0.   0.   0.1  0.   0.4 -0.   0.   0.   0.1
   0.   0.   0.   0.  -0.   0.1  0.1]
 [43.6 12.1  1.   0.9 -0.  -0.  -0.   0.   1.  -0.  -0.  -0.   0.   0.1
  -0.   0.9  0.   0.   0.   0.  -0.   0.   0.   0.   1.  -0.  -0.  -0.
  -0.   0.   0.  -0.  -0.  -0.   0. ]
 [44.1 13.6  0.9  0.8  0.   0.   0.2  0.   0.5  0.1  0.1  0.1  0.   0.1
   0.   0.8  0.   0.1  0.   0.   0.  -0.   0.   0.  -0.  -0.  -0.  -0.
  -0.   0.   1.  -0.  -0.   0.   0. ]
 [46.1 10.4  0.9  0.8  0.2  0.2  0.3  0.   0.   0.1  0.2  0.1  0.   0.1
  -0.   0.9  0.   0.   0.   0.   0.2  0.2  0.   0.2 -0.   0.1  0.   0.
   0.   0.   0.   0.2 -0.   0.   0.1]
 [43.8 11.6  0.9  0.9 -0.   0.  -0.   0.   0.7  0.2  0.1 -0.   0.   0.
  -0.   0.9 -0.   0.1  0

In [41]:
x_s = pickle.load(open("../data/adult/kmeans_x_s.pkl", "rb"))
x_s_group = pickle.load(open("../data/adult/kmeans_x_s_group.pkl", "rb"))
np.random.seed(0)
subset = np.random.permutation(source.shape[0])[:1000]

In [42]:
from sklearn.model_selection import train_test_split
from src.training import eval_acc

# Training an embedding model
adult_emb = PTNN(35, 1)
no_group = torch.ones((source.shape[0] + target.shape[0], 1))
X = torch.from_numpy(np.concatenate([source, target], axis=0)).float()
y = torch.from_numpy(np.concatenate([np.ones((source.shape[0], 1)), np.zeros((target.shape[0], 1))], axis=0)).float()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
regular_training(adult_emb,
                 X_train,
                 y_train,
                 torch.ones_like(y_train), 100, 0.5, 5e-5)
eval_acc(adult_emb, X_test, y_test, torch.ones_like(y_test))

tensor(0.5023)


0 0.7008904218673706
0 0.697862982749939
0 0.6938037276268005
0 0.6935368776321411
0 0.6959402561187744
0 0.6927169561386108
0 0.6965129971504211
0 0.6908556222915649
0 0.691049337387085
0 0.6907034516334534
0 0.6902616024017334
0 0.6896260380744934
0 0.6885135173797607
1 0.6851104497909546
1 0.6820650100708008
1 0.6807675361633301
1 0.6743819713592529
1 0.661581814289093
1 0.6533207297325134
1 0.630277156829834
1 0.6264639496803284
1 0.5933193564414978
1 0.5714436769485474
1 0.5815576314926147
1 0.5574612021446228
1 0.6020534634590149
2 0.5493739247322083
2 0.5639229416847229
2 0.5439099669456482
2 0.5222582221031189
2 0.5200270414352417
2 0.5277186632156372
2 0.5402382016181946
2 0.5113329291343689
2 0.501491904258728
2 0.5111237168312073
2 0.5146939754486084
2 0.5293300151824951
2 0.4714912176132202
3 0.5042356848716736
3 0.5053307414054871
3 0.5116676092147827
3 0.4981105327606201
3 0.47723641991615295
3 0.4646274149417877
3 0.48996084928512573
3 0.521593451499939
3 0.4887116849422

28 0.45108380913734436
28 0.4513998031616211
28 0.41615283489227295
29 0.44606342911720276
29 0.48463109135627747
29 0.43236243724823
29 0.420337051153183
29 0.45231783390045166
29 0.4507986009120941
29 0.4358780086040497
29 0.4291718900203705
29 0.4239436686038971
29 0.4445519745349884
29 0.4726620018482208
29 0.4379827380180359
29 0.44725099205970764
30 0.46551334857940674
30 0.427514910697937
30 0.4122748076915741
30 0.43866315484046936
30 0.4467437267303467
30 0.4490642845630646
30 0.4403410851955414
30 0.4458656311035156
30 0.4527687430381775
30 0.43761566281318665
30 0.442756325006485
30 0.4269769489765167
30 0.4288906455039978
31 0.46124130487442017
31 0.4668377637863159
31 0.43912386894226074
31 0.44723549485206604
31 0.4279077649116516
31 0.4249795079231262
31 0.4487452507019043
31 0.4106709659099579
31 0.4317648112773895
31 0.4514465630054474
31 0.4263747036457062
31 0.41482600569725037
31 0.42793378233909607
32 0.44936835765838623
32 0.4442943036556244
32 0.4194924533367157


58 0.47176939249038696
58 0.4509356915950775
58 0.44891592860221863
58 0.44750991463661194
58 0.43707698583602905
58 0.4261481463909149
58 0.40515753626823425
58 0.4318822920322418
59 0.44660109281539917
59 0.42187222838401794
59 0.437804251909256
59 0.4349514842033386
59 0.4537040889263153
59 0.4363005459308624
59 0.4419947862625122
59 0.4504462480545044
59 0.43555495142936707
59 0.4038797616958618
59 0.4215177595615387
59 0.4242135286331177
59 0.40789228677749634
60 0.42189186811447144
60 0.4035448729991913
60 0.4571518301963806
60 0.40996408462524414
60 0.43236950039863586
60 0.42553213238716125
60 0.436940997838974
60 0.4509230852127075
60 0.42313361167907715
60 0.45093244314193726
60 0.4306030571460724
60 0.4755517840385437
60 0.4180958569049835
61 0.4429894685745239
61 0.41752463579177856
61 0.4754614233970642
61 0.4234621822834015
61 0.4158692955970764
61 0.3982403576374054
61 0.4164777994155884
61 0.4433944821357727
61 0.4489184319972992
61 0.40614157915115356
61 0.440098643302

88 0.42685946822166443
88 0.4493585526943207
88 0.45396867394447327
88 0.4415600895881653
88 0.42149850726127625
88 0.4322497248649597
88 0.4055424928665161
88 0.40513184666633606
89 0.41934189200401306
89 0.43200331926345825
89 0.46659860014915466
89 0.41253554821014404
89 0.429512619972229
89 0.44357559084892273
89 0.42122799158096313
89 0.43352946639060974
89 0.4463936984539032
89 0.41369450092315674
89 0.44561174511909485
89 0.4144015610218048
89 0.4385199248790741
90 0.4158420264720917
90 0.45210543274879456
90 0.4430398941040039
90 0.4637013077735901
90 0.43464165925979614
90 0.4536440968513489
90 0.4465092420578003
90 0.41263478994369507
90 0.44224563241004944
90 0.4049485921859741
90 0.419332355260849
90 0.43675369024276733
90 0.40120840072631836
91 0.4500744640827179
91 0.4459473490715027
91 0.456805557012558
91 0.42554518580436707
91 0.4353041648864746
91 0.41407907009124756
91 0.3949258625507355
91 0.4342533349990845
91 0.4086450934410095
91 0.42282038927078247
91 0.44930216

torch.Size([12545, 1]) torch.Size([12545, 1])
Accuracy: tensor(0.8151)
tensor([0.8151])
Accuracy: tensor(0.7969)
tensor([0.7969])


In [44]:
from src.distance import percent_flipped

percent_flipped(adult_emb, source[subset], x_s, source_feasible_groups[subset])
percent_flipped(adult_emb, source[subset], x_s_group, source_feasible_groups[subset])

62.8 [64. 56.]
58.5 [58.47058824 58.66666667]


In [265]:
with torch.no_grad():
    adult_emb.eval()
    source_emb = adult_emb.embed(torch.from_numpy(source[subset]).float()).detach().numpy()
    source_t_emb = adult_emb.embed(torch.from_numpy(x_s).float()).numpy()
    source_t_group_emb = adult_emb.embed(torch.from_numpy(x_s_group).float()).numpy()
    target_emb = adult_emb.embed(torch.from_numpy(target[subset]).float()).detach().numpy()
    

group_percent_explained(
    source_emb,
    source_t_emb,
    target_emb,
    source_feasible_groups[subset],
    target_feasible_groups[subset],
    feasible_names)

group_percent_explained(
    source_emb,
    source_t_group_emb,
    target_emb,
    source_feasible_groups[subset],
    target_feasible_groups[subset],
    feasible_names)

Total Percent Explained: 89.85261426740823
male Percent Explained: 92.10103401362834
female Percent Explained: 76.79101416097328
Worst group PE: 76.79101416097328
Total Percent Explained: 84.21785807971894
male Percent Explained: 84.92925585478581
female Percent Explained: 97.70257268923604
Worst group PE: 84.92925585478581


(84.21785807971894, 84.92925585478581)

## Breast Cancer

In [25]:
from sklearn.preprocessing import StandardScaler

source = bc_source
target = bc_target
source_feasible_groups = bc_source_feasible_groups
target_feasible_groups = bc_target_feasible_groups
feasible_names = bc_feasible_names
scaler = bc_scaler
is_feasible_change = is_bc_feasible_change

In [26]:
# Training an embedding model
bc_emb = PTNN(10, 1)
regular_training(bc_emb,
                 torch.from_numpy(np.concatenate([source, target], axis=0)).float(),
                 torch.from_numpy(np.concatenate([np.ones((source.shape[0], 1)), np.zeros((target.shape[0], 1))], axis=0)).float(),
                 torch.ones((source.shape[0] + target.shape[0], 1)), 1000, 0.05, 1e-3)

tensor(0.5000)


0 0.6986159682273865
1 0.6968143582344055
2 0.6935056447982788
3 0.6885269284248352
4 0.6893822550773621
5 0.6817744970321655
6 0.6781224012374878
7 0.6664660573005676
8 0.6614444851875305
9 0.6539256572723389
10 0.6471483707427979
11 0.6345577239990234
12 0.6226862668991089
13 0.605462908744812
14 0.5888398289680481
15 0.5677241683006287
16 0.5487483143806458
17 0.5189812779426575
18 0.4928702414035797
19 0.4595407545566559
20 0.4383414387702942
21 0.40548956394195557
22 0.3641142249107361
23 0.3445449471473694
24 0.33266931772232056
25 0.3303019106388092
26 0.2825350761413574
27 0.31666508316993713
28 0.3086189031600952
29 0.26212602853775024
30 0.23361879587173462
31 0.22756464779376984
32 0.23094789683818817
33 0.23320892453193665
34 0.25710803270339966
35 0.2386074811220169
36 0.1883794218301773
37 0.2319515198469162
38 0.21019764244556427
39 0.2192763239145279
40 0.23473073542118073
41 0.25337880849838257
42 0.2825084924697876
43 0.2232658565044403
44 0.21822257339954376
45 0.224

360 0.09608864784240723
361 0.08427216112613678
362 0.08224125951528549
363 0.08524218201637268
364 0.10979938507080078
365 0.0957883894443512
366 0.07971624284982681
367 0.09634844958782196
368 0.09550260007381439
369 0.09597356617450714
370 0.09804873168468475
371 0.07625532150268555
372 0.08777269721031189
373 0.09325463324785233
374 0.09114040434360504
375 0.08018927276134491
376 0.08898218721151352
377 0.10413587838411331
378 0.08551674336194992
379 0.0924304872751236
380 0.08782404661178589
381 0.11455048620700836
382 0.08365513384342194
383 0.09755115956068039
384 0.1065167710185051
385 0.09506650269031525
386 0.09149371832609177
387 0.08731186389923096
388 0.09392762184143066
389 0.0980362817645073
390 0.08963532745838165
391 0.07781010866165161
392 0.10328289121389389
393 0.08996380865573883
394 0.08154062926769257
395 0.0977374017238617
396 0.08893600851297379
397 0.0772564709186554
398 0.08753814548254013
399 0.08032478392124176
400 0.09394478797912598
401 0.0910860374569892

726 0.05410296097397804
727 0.051417261362075806
728 0.0850687325000763
729 0.05336053669452667
730 0.05278245359659195
731 0.055551134049892426
732 0.044764913618564606
733 0.08034723252058029
734 0.058841463178396225
735 0.04394770786166191
736 0.06353133916854858
737 0.07569295167922974
738 0.054026152938604355
739 0.05743910372257233
740 0.061795156449079514
741 0.051605287939310074
742 0.06496229022741318
743 0.05621242895722389
744 0.06961778551340103
745 0.06349138170480728
746 0.05422564968466759
747 0.0764661505818367
748 0.055111102759838104
749 0.051108598709106445
750 0.05935506522655487
751 0.05176384001970291
752 0.05355037748813629
753 0.056336697190999985
754 0.055244363844394684
755 0.04762628301978111
756 0.06326514482498169
757 0.05181339755654335
758 0.05764872580766678
759 0.06175016611814499
760 0.04903214052319527
761 0.05658532306551933
762 0.053123511373996735
763 0.0565638467669487
764 0.059780530631542206
765 0.04462660849094391
766 0.06531653553247452
767 0.

torch.Size([424, 1]) torch.Size([424, 1])
Accuracy: tensor(0.9929)
tensor([0.9929])


In [40]:
from src.optimal_transport import group_mean_shift_transport, group_kmeans_shift_transport, transform_samples_kmeans
from src.distance import group_percent_explained
import pickle

reg_total = []
reg_worst = []
reg_feas = []
reg_pf = []
reg_pf_worst = []
group_total = []
group_worst = []
group_feas = []
group_pf = []
group_pf_worst = []
for i in range(3):
    np.random.seed(i)
    no_group_s = np.ones((source.shape[0], 1))
    no_group_t = np.ones((target.shape[0], 1))
    x_s, centroids, shifts = group_kmeans_shift_transport(source, target, no_group_s, no_group_t, source.shape[1], 4, 1.0, 100)
    total, worst = group_percent_explained(
        source,
        x_s,
        target,
        source_feasible_groups,
        target_feasible_groups,
        feasible_names)

    feasible = is_feasible_change(source, x_s)
    reg_feas.append(np.sum(feasible) / feasible.shape[0])
    reg_total.append(total)
    reg_worst.append(worst)

    x_s_group, centroids, shifts = group_kmeans_shift_transport(source, target, source_feasible_groups, target_feasible_groups, source.shape[1], 4, 1.0, 100)
    g_total, g_worst = group_percent_explained(
        source,
        x_s_group,
        target,
        source_feasible_groups,
        target_feasible_groups,
        feasible_names)
    group_total.append(g_total)
    group_worst.append(g_worst)
    
    g_feasible = is_feasible_change(source, x_s_group)
    group_feas.append(np.sum(feasible) / feasible.shape[0])

    pf, pf_worst = percent_flipped(bc_emb, source, x_s, source_feasible_groups)
    reg_pf.append(pf)
    reg_pf_worst.append(pf_worst)
    pf, pf_worst = percent_flipped(bc_emb, source, x_s_group, source_feasible_groups)
    group_pf.append(pf)
    group_pf_worst.append(pf_worst)

Samples in cluster 0: 17
Samples in cluster 1: 79
Samples in cluster 2: 38
Samples in cluster 3: 78
Iter 0 loss: 1.0
Change in x_s: 0.30415183305740356
Iter 1 loss: 0.9105732780743742
Change in x_s: 0.28351446986198425
Iter 2 loss: 0.8329872669357871
Change in x_s: 0.2649628221988678
Iter 3 loss: 0.7657041845487045
Change in x_s: 0.2448054850101471
Iter 4 loss: 0.7081144279435908
Change in x_s: 0.2264305055141449
Iter 5 loss: 0.658633871574235
Change in x_s: 0.2090727686882019
Iter 6 loss: 0.6158622282721911
Change in x_s: 0.19800984859466553
Iter 7 loss: 0.5780083230068701
Change in x_s: 0.18286627531051636
Iter 8 loss: 0.5455602628383256
Change in x_s: 0.173080176115036
Iter 9 loss: 0.516942394062791
Change in x_s: 0.1603815108537674
Iter 10 loss: 0.4921812655974276
Change in x_s: 0.15176835656166077
Iter 11 loss: 0.470232680324974
Change in x_s: 0.14269192516803741
Iter 12 loss: 0.4509352402408953
Change in x_s: 0.1325637549161911
Iter 13 loss: 0.43418329023930785
Change in x_s: 0.1

Iter 13 loss: 0.4754599379461847
Change in x_s: 0.11086691170930862
Iter 14 loss: 0.4636137582688472
Change in x_s: 0.10270407050848007
Iter 15 loss: 0.4533751266364778
Change in x_s: 0.09635438770055771
Iter 16 loss: 0.44437619484107604
Change in x_s: 0.0901726707816124
Iter 17 loss: 0.4365171544525977
Change in x_s: 0.08464185893535614
Iter 18 loss: 0.42958352871711464
Change in x_s: 0.07967575639486313
Iter 19 loss: 0.4234578490653197
Change in x_s: 0.07487121224403381
Iter 20 loss: 0.41805164558234686
Change in x_s: 0.07047120481729507
Iter 21 loss: 0.4132735342388398
Change in x_s: 0.06615795195102692
Iter 22 loss: 0.40905559431091826
Change in x_s: 0.06249256432056427
Iter 23 loss: 0.4052882327910531
Change in x_s: 0.05936408415436745
Iter 24 loss: 0.40190510957412967
Change in x_s: 0.05615397170186043
Iter 25 loss: 0.3988618780627485
Change in x_s: 0.05350540205836296
Iter 26 loss: 0.3961162370869079
Change in x_s: 0.05075496807694435
Iter 27 loss: 0.3936465289324287
Change in x

Iter 25 loss: 0.355863401399659
Change in x_s: 0.05562157183885574
Iter 26 loss: 0.3528827722169382
Change in x_s: 0.05574396252632141
Iter 27 loss: 0.34999608298921814
Change in x_s: 0.051519040018320084
Iter 28 loss: 0.3475224082963234
Change in x_s: 0.047771479934453964
Iter 29 loss: 0.34534707497395484
Change in x_s: 0.046999067068099976
Iter 30 loss: 0.34327190676092223
Change in x_s: 0.04490269720554352
Iter 31 loss: 0.34129020128605503
Change in x_s: 0.04511820152401924
Iter 32 loss: 0.33933208456379355
Change in x_s: 0.04390416294336319
Iter 33 loss: 0.33752469414064584
Change in x_s: 0.04188835620880127
Iter 34 loss: 0.3358412314713906
Change in x_s: 0.0419110506772995
Iter 35 loss: 0.3341957533961521
Change in x_s: 0.03902469202876091
Iter 36 loss: 0.3327452488020816
Change in x_s: 0.0368533730506897
Iter 37 loss: 0.33139169380237327
Change in x_s: 0.03695991635322571
Iter 38 loss: 0.3300962466021969
Change in x_s: 0.034898463636636734
Iter 39 loss: 0.32889274458938034
Change

Iter 39 loss: 0.3931477331246539
Change in x_s: 0.0273843202739954
Iter 40 loss: 0.392463543992979
Change in x_s: 0.026203813031315804
Iter 41 loss: 0.39183328511541904
Change in x_s: 0.025336438789963722
Iter 42 loss: 0.3912330308147563
Change in x_s: 0.02515232004225254
Iter 43 loss: 0.3906439514500467
Change in x_s: 0.025473253801465034
Iter 44 loss: 0.39005356082397735
Change in x_s: 0.024511201307177544
Iter 45 loss: 0.3895099789099895
Change in x_s: 0.023538311943411827
Iter 46 loss: 0.3890132124474198
Change in x_s: 0.022716620936989784
Iter 47 loss: 0.3885520747925544
Change in x_s: 0.0222835224121809
Iter 48 loss: 0.38811715256140017
Change in x_s: 0.021555250510573387
Iter 49 loss: 0.3877146287531701
Change in x_s: 0.020897554233670235
Iter 50 loss: 0.3873413877234224
Change in x_s: 0.020157430320978165
Iter 51 loss: 0.3869958536049366
Change in x_s: 0.01961008831858635
Iter 52 loss: 0.3866701931393472
Change in x_s: 0.019199619069695473
Iter 53 loss: 0.3863626430900232
Chang

Iter 52 loss: 0.32643683151262365
Change in x_s: 0.023156899958848953
Iter 53 loss: 0.32591209688221373
Change in x_s: 0.02196686528623104
Iter 54 loss: 0.32543690203383996
Change in x_s: 0.020899120718240738
Iter 55 loss: 0.3250053593742618
Change in x_s: 0.019908424466848373
Iter 56 loss: 0.32461230008400926
Change in x_s: 0.018998214974999428
Iter 57 loss: 0.3242527442797064
Change in x_s: 0.018229976296424866
Iter 58 loss: 0.32391369224564504
Change in x_s: 0.018872590735554695
Iter 59 loss: 0.3235634685466865
Change in x_s: 0.01819026470184326
Iter 60 loss: 0.3232323137849053
Change in x_s: 0.01866154372692108
Iter 61 loss: 0.3228940353142158
Change in x_s: 0.01923820748925209
Iter 62 loss: 0.32254282786367955
Change in x_s: 0.019364764913916588
Iter 63 loss: 0.3221682102875362
Change in x_s: 0.019989896565675735
Iter 64 loss: 0.32178911223120665
Change in x_s: 0.019225677475333214
Iter 65 loss: 0.3214415233945794
Change in x_s: 0.018331265076994896
Iter 66 loss: 0.321125642247733

Iter 66 loss: 0.3487510318119529
Change in x_s: 0.018427694216370583
Iter 67 loss: 0.3484890365544042
Change in x_s: 0.01842796429991722
Iter 68 loss: 0.3482292068316527
Change in x_s: 0.018000060692429543
Iter 69 loss: 0.3479800580079301
Change in x_s: 0.01779046840965748
Iter 70 loss: 0.3477424823367381
Change in x_s: 0.01728246547281742
Iter 71 loss: 0.34752237529722907
Change in x_s: 0.01665150187909603
Iter 72 loss: 0.3473243290980479
Change in x_s: 0.01627088338136673
Iter 73 loss: 0.3471303015719459
Change in x_s: 0.016231907531619072
Iter 74 loss: 0.34694727914842494
Change in x_s: 0.015581442043185234
Iter 75 loss: 0.34678473326643455
Change in x_s: 0.0150916688144207
Iter 76 loss: 0.34661079808223727
Change in x_s: 0.017234794795513153
Iter 77 loss: 0.34639551907188354
Change in x_s: 0.017200347036123276
Iter 78 loss: 0.3461756138793775
Change in x_s: 0.017195580527186394
Iter 79 loss: 0.345964028684276
Change in x_s: 0.01643965020775795
Iter 80 loss: 0.34575811827092584
Chan

In [41]:
print(np.mean(reg_total), np.std(reg_total))
print(np.mean(reg_worst), np.std(reg_worst))
print(np.mean(reg_feas), np.std(reg_feas))
print(np.mean(reg_pf), np.std(reg_pf))
print(np.mean(reg_pf_worst), np.std(reg_pf_worst))
print()
print(np.mean(group_total), np.std(group_total))
print(np.mean(group_worst), np.std(group_worst))
print(np.mean(group_feas), np.std(group_feas))
print(np.mean(group_pf), np.std(group_pf))
print(np.mean(group_pf_worst), np.std(group_pf_worst))

85.35409504467377 0.3151221697664498
53.31020171223647 5.539830516437766
0.5849056603773585 0.037141546575527415
96.0691823899371 0.8017326279233951
92.45283018867924 0.0

84.53954754991172 0.7109135353716731
73.14838177489519 0.3708873780416552
0.5849056603773585 0.037141546575527415
97.64150943396226 0.0
92.45283018867924 0.0


In [48]:
centroids_i = scaler.inverse_transform(centroids)
centroids_f = scaler.inverse_transform(centroids + shifts)
shifts = centroids_f - centroids_i
feasible = is_feasible_change(source, x_s)
print("Percent feasible shift:", (np.sum(feasible) / feasible.shape[0]) * 100)

Percent feasible shift: 61.32075471698113


In [49]:
print(centroids_i.round(1))
print((shifts).round(1))
for shift in shifts:
    print([f"{bc_feature_names[i]}: {s:.2f}" for i, s in enumerate(shift.round(1)) if s != 0])

[[0.4 0.6 0.4 0.2 0.6 0.2 0.1 0.1 0.5 0.6]
 [0.4 0.4 0.4 0.2 0.7 0.3 0.1 0.2 0.6 0.7]
 [0.4 0.4 0.4 0.2 0.6 0.2 0.1 0.1 0.5 0.6]
 [0.4 0.4 0.4 0.1 0.7 0.4 0.3 0.2 0.7 0.8]]
[[ 0.1 -0.   0.1  0.1  0.1  0.1  0.1  0.2  0.  -0. ]
 [ 0.2  0.1  0.2  0.3  0.1  0.3  0.4  0.4  0.1  0. ]
 [ 0.2  0.1  0.2  0.2  0.1  0.2  0.2  0.3  0.  -0. ]
 [ 0.1  0.   0.1  0.1  0.1  0.1  0.1  0.1  0.  -0. ]]
['radius_mean: 0.10', 'perimeter_mean: 0.10', 'area_mean: 0.10', 'smoothness_mean: 0.10', 'compactness_mean: 0.10', 'concavity_mean: 0.10', 'concave points_mean: 0.20']
['radius_mean: 0.20', 'texture_mean: 0.10', 'perimeter_mean: 0.20', 'area_mean: 0.30', 'smoothness_mean: 0.10', 'compactness_mean: 0.30', 'concavity_mean: 0.40', 'concave points_mean: 0.40', 'symmetry_mean: 0.10']
['radius_mean: 0.20', 'texture_mean: 0.10', 'perimeter_mean: 0.20', 'area_mean: 0.20', 'smoothness_mean: 0.10', 'compactness_mean: 0.20', 'concavity_mean: 0.20', 'concave points_mean: 0.30']
['radius_mean: 0.10', 'perimeter_mean: 0

In [76]:
from src.optimal_transport import transform_samples_kmeans
import pickle

# subset = np.random.permutation(source.shape[0])[:source.shape[0]//10]
x_s_group, centroids, shifts = group_kmeans_shift_transport(source, target, source_feasible_groups, target_feasible_groups, source.shape[1], 4, 1.0, 100)
# x_s = transform_samples_kmeans(source, centroids, shifts)
group_percent_explained(source, x_s_group, target, source_feasible_groups, target_feasible_groups, feasible_names)

centroids_i = scaler.inverse_transform(centroids)
centroids_f = scaler.inverse_transform(centroids + shifts)
shifts = centroids_f - centroids_i

with open("../data/breast_cancer/kmeans_x_s_group.pkl", "wb") as f:
    pickle.dump(x_s_group, f)

Samples in cluster 0: 39
Samples in cluster 1: 83
Samples in cluster 2: 74
Samples in cluster 3: 16
Iter 0 loss: 1.0
Iter 1 loss: 0.9443019742820071
Iter 2 loss: 0.8920215503823268
Iter 3 loss: 0.8428140137888518
Iter 4 loss: 0.7969251493351882
Iter 5 loss: 0.7544712397687289
Iter 6 loss: 0.7149515187926595
Iter 7 loss: 0.6781383741658367
Iter 8 loss: 0.643821498361014
Iter 9 loss: 0.6119756517951893
Iter 10 loss: 0.5821950800853595
Iter 11 loss: 0.5543860032314071
Iter 12 loss: 0.5286530744321571
Iter 13 loss: 0.5047855105749798
Iter 14 loss: 0.48275183698961827
Iter 15 loss: 0.46229943105561977
Iter 16 loss: 0.4431773363132102
Iter 17 loss: 0.4253228634259606
Iter 18 loss: 0.408787473769364
Iter 19 loss: 0.393477139854537
Iter 20 loss: 0.37928952071270744
Iter 21 loss: 0.36621675373992124
Iter 22 loss: 0.3540732114599579
Iter 23 loss: 0.3427879304079195
Iter 24 loss: 0.33239359818030934
Iter 25 loss: 0.32278727225461795
Iter 26 loss: 0.3139289367413052
Iter 27 loss: 0.305702337372328

In [51]:
feasible = is_feasible_change(source, x_s_group)
print("Percent feasible shift:", (np.sum(feasible) / feasible.shape[0]) * 100)
for shift in shifts:
    print([f"{bc_feature_names[i]}: {s:.2f}" for i, s in enumerate(shift.round(1)) if s != 0])

Percent feasible shift: 62.264150943396224
['radius_mean: 0.10', 'perimeter_mean: 0.10', 'area_mean: 0.10', 'smoothness_mean: 0.10', 'compactness_mean: 0.10', 'concavity_mean: 0.10', 'concave points_mean: 0.20']
['radius_mean: 0.20', 'texture_mean: 0.10', 'perimeter_mean: 0.20', 'area_mean: 0.30', 'smoothness_mean: 0.10', 'compactness_mean: 0.30', 'concavity_mean: 0.40', 'concave points_mean: 0.40', 'symmetry_mean: 0.10']
['radius_mean: 0.20', 'texture_mean: 0.10', 'perimeter_mean: 0.20', 'area_mean: 0.20', 'smoothness_mean: 0.10', 'compactness_mean: 0.20', 'concavity_mean: 0.20', 'concave points_mean: 0.30']
['radius_mean: 0.10', 'perimeter_mean: 0.10', 'area_mean: 0.10', 'smoothness_mean: 0.10', 'compactness_mean: 0.10', 'concavity_mean: 0.10', 'concave points_mean: 0.10']


In [74]:
# Training an embedding model
bc_emb = PTNN(10, 1)
regular_training(bc_emb,
                 torch.from_numpy(np.concatenate([source, target], axis=0)).float(),
                 torch.from_numpy(np.concatenate([np.ones((source.shape[0], 1)), np.zeros((target.shape[0], 1))], axis=0)).float(),
                 torch.ones((source.shape[0] + target.shape[0], 1)), 1000, 0.05, 1e-3)

tensor(0.5000)


0 0.6954307556152344
1 0.6938355565071106
2 0.6941938400268555
3 0.6904160976409912
4 0.6907725930213928
5 0.6895067095756531
6 0.6831793785095215
7 0.678297758102417
8 0.6762052178382874
9 0.6705124378204346
10 0.6649929881095886
11 0.6618245244026184
12 0.6536775827407837
13 0.6442195177078247
14 0.6353576183319092
15 0.626361072063446
16 0.6101978421211243
17 0.5799484252929688
18 0.5637086033821106
19 0.5441144704818726
20 0.5246410965919495
21 0.4975314736366272
22 0.4708234667778015
23 0.4318702220916748
24 0.4020118713378906
25 0.37401488423347473
26 0.34353840351104736
27 0.33368051052093506
28 0.2978668212890625
29 0.3143846392631531
30 0.2820982336997986
31 0.2610670328140259
32 0.2320735603570938
33 0.2843221426010132
34 0.2336309403181076
35 0.24798835813999176
36 0.23530419170856476
37 0.24413643777370453
38 0.2140417993068695
39 0.2314428836107254
40 0.2171449512243271
41 0.22048123180866241
42 0.20407313108444214
43 0.23170046508312225
44 0.2519949972629547
45 0.21268711

372 0.0829751119017601
373 0.07604008167982101
374 0.08624965697526932
375 0.08738373965024948
376 0.10443323850631714
377 0.08626130223274231
378 0.08706226944923401
379 0.09265374392271042
380 0.08132939040660858
381 0.09529808163642883
382 0.07567012310028076
383 0.08062610775232315
384 0.09506616741418839
385 0.0887332409620285
386 0.07622256129980087
387 0.09565409272909164
388 0.08250267058610916
389 0.10808960348367691
390 0.08897120505571365
391 0.1026521548628807
392 0.10784950107336044
393 0.08420611172914505
394 0.08642780035734177
395 0.09278886020183563
396 0.08538100868463516
397 0.07442488521337509
398 0.09862691164016724
399 0.0948835089802742
400 0.08397532254457474
401 0.08252307772636414
402 0.1031971201300621
403 0.08485211431980133
404 0.08495239168405533
405 0.07928435504436493
406 0.07197361439466476
407 0.0839916244149208
408 0.0699491947889328
409 0.0858171135187149
410 0.0790378674864769
411 0.08378608524799347
412 0.07432222366333008
413 0.09841953963041306
4

759 0.06060628592967987
760 0.06780384480953217
761 0.05007784441113472
762 0.053396113216876984
763 0.0548841692507267
764 0.048261821269989014
765 0.04842784255743027
766 0.04060809686779976
767 0.05134541541337967
768 0.05480964481830597
769 0.04512336105108261
770 0.05688788741827011
771 0.06294692307710648
772 0.06127838045358658
773 0.04761315509676933
774 0.0598655566573143
775 0.04586756229400635
776 0.05369246006011963
777 0.062436170876026154
778 0.07941486686468124
779 0.05566525459289551
780 0.04419289156794548
781 0.05180911719799042
782 0.05403507873415947
783 0.047059133648872375
784 0.048063505440950394
785 0.04423171654343605
786 0.04298800975084305
787 0.049920015037059784
788 0.054886702448129654
789 0.046681154519319534
790 0.06287816166877747
791 0.060441676527261734
792 0.057563360780477524
793 0.05507392808794975
794 0.05029352009296417
795 0.051605675369501114
796 0.06457984447479248
797 0.060537468641996384
798 0.053303156048059464
799 0.04712555557489395
800 0

torch.Size([424, 1]) torch.Size([424, 1])
Accuracy: tensor(0.9929)
tensor([0.9929])


In [72]:
x_s = pickle.load(open("../data/breast_cancer/kmeans_x_s.pkl", "rb"))
x_s_group = pickle.load(open("../data/breast_cancer/kmeans_x_s_group.pkl", "rb"))

In [77]:
from src.distance import percent_flipped

percent_flipped(bc_emb, source, x_s, source_feasible_groups)
percent_flipped(bc_emb, source, x_s_group, source_feasible_groups)

91.50943396226415 [81.13207547 94.33962264 96.22641509]
91.0377358490566 [81.13207547 93.39622642 96.22641509]


In [54]:
with torch.no_grad():
    bc_emb.eval()
    source_emb = bc_emb.embed(torch.from_numpy(source).float()).detach().numpy()
    source_t_emb = bc_emb.embed(torch.from_numpy(x_s).float()).numpy()
    source_t_group_emb = bc_emb.embed(torch.from_numpy(x_s_group).float()).numpy()
    target_emb = bc_emb.embed(torch.from_numpy(target).float()).detach().numpy()
    
group_percent_explained(
    source_emb,
    source_t_emb,
    target_emb,
    source_feasible_groups,
    target_feasible_groups,
    feasible_names)

group_percent_explained(
    source_emb,
    source_t_group_emb,
    target_emb,
    source_feasible_groups,
    target_feasible_groups,
    feasible_names)

Total Percent Explained: 96.5096179580727
radius**2 / area above third quartile Percent Explained: 91.62424475364394
radius**2 / area between first and third quartile Percent Explained: 97.16955311739324
radius**2 / area below first quartile Percent Explained: 91.32687441641554
Worst group PE: 91.32687441641554
Total Percent Explained: 96.10985387204812
radius**2 / area above third quartile Percent Explained: 91.11174634698435
radius**2 / area between first and third quartile Percent Explained: 96.75897129535859
radius**2 / area below first quartile Percent Explained: 92.11883073519358
Worst group PE: 91.11174634698435


(96.10985387204812, 91.11174634698435)

# Worst Group OT

## Adult

### Non-group regularized results

In [42]:
np.random.seed(0)
subset = np.random.permutation(1000) #adult_source.shape[0])
subset_train = subset[:int(subset.shape[0] * 0.6)]
subset_test = subset[int(subset.shape[0] * 0.6):]

In [43]:
from sklearn.ensemble import GradientBoostingClassifier

X_train = np.concatenate([adult_source[subset_test, :], adult_target[subset_test, :]])
y_train = np.concatenate([adult_source_labels[subset_test, 0], adult_target_labels[subset_test, 0]])
clf = GradientBoostingClassifier(n_estimators=100).fit(X_train, y_train)

In [45]:
from src.optimal_transport import group_feature_transport, transform_samples
import pickle

np.random.seed(0)
subset = np.random.permutation(1000)
subset_train = subset[:int(subset.shape[0] * 0.6)]
subset_test = subset[int(subset.shape[0] * 0.6):]

no_group_s = np.ones((adult_source.shape[0], 1))
no_group_t = np.ones((adult_target.shape[0], 1))

reg_total = []
reg_worst = []
reg_feas = []
reg_pf = []
reg_pf_worst = []
group_total = []
group_worst = []
group_feas = []
group_pf = []
group_pf_worst = []
for i in range(3):
    np.random.seed(i * 1000)
    x_s_train = group_feature_transport(adult_source[subset_train, :], adult_target[subset_train, :], no_group_s[subset_train, :], no_group_t[subset_train, :], 36, 0.5, 100)

    total, worst = group_percent_explained(
        adult_source[subset_train, :],
        adult_scaler.transform(adult_scaler.inverse_transform(x_s_train).round(0)),
        adult_target[subset_train, :],
        adult_source_feasible_groups[subset_train, :],
        adult_target_feasible_groups[subset_train, :],
        adult_feasible_names)
    reg_total.append(total)
    reg_worst.append(worst)

    pf, pf_worst = percent_flipped(lambda x: torch.from_numpy(clf.predict(x)), adult_source[subset_train, :], x_s_train, adult_source_feasible_groups[subset_train, :])
    reg_pf.append(pf)
    reg_pf_worst.append(pf_worst)

    feasible_train = is_adult_feasible_change(
        adult_scaler.inverse_transform(adult_source[subset_train, :]).round(0),
        adult_scaler.inverse_transform(x_s_train).round(0))
    reg_feas.append(np.sum(feasible_train) / feasible_train.shape[0])

[0]
Iter 0 loss: 1.0
Change in x_s: 14.009062767028809
Iter 1 loss: 0.4437984960655949
Change in x_s: 10.001280784606934
Iter 2 loss: 0.176514258639158
Change in x_s: 6.155865669250488
Iter 3 loss: 0.07480913769840959
Change in x_s: 3.6967833042144775
Iter 4 loss: 0.03894461657368521
Change in x_s: 2.1070504188537598
Iter 5 loss: 0.027541787004355132
Change in x_s: 1.2296842336654663
Iter 6 loss: 0.023694010988102476
Change in x_s: 0.8093134164810181
Iter 7 loss: 0.021914433401679874
Change in x_s: 0.5488517880439758
Iter 8 loss: 0.021109807452837064
Change in x_s: 0.39404407143592834
Iter 9 loss: 0.020611707385673028
Change in x_s: 0.535722553730011
Iter 10 loss: 0.01973854501329752
Change in x_s: 0.388387531042099
Iter 11 loss: 0.019348040416009838
Change in x_s: 0.22686323523521423
Iter 12 loss: 0.01921829245312101
Change in x_s: 0.1340044140815735
Iter 13 loss: 0.019177054499393695
Change in x_s: 0.08299209177494049
Iter 14 loss: 0.019145883711148837
Change in x_s: 0.06646378338336

Iter 45 loss: 0.02000084367507691
Change in x_s: 0.009924869984388351
Iter 46 loss: 0.02000082492338602
Change in x_s: 0.009478716179728508
Iter 47 loss: 0.020000820516267108
Change in x_s: 0.00906810536980629
Iter 48 loss: 0.020000829424040623
Change in x_s: 0.008688971400260925
Iter 49 loss: 0.020000843175127167
Change in x_s: 0.008338488638401031
Iter 50 loss: 0.020000865649305765
Change in x_s: 0.008014686405658722
Iter 51 loss: 0.02000089555191366
Change in x_s: 0.007715932559221983
Iter 52 loss: 0.020000931080952373
Change in x_s: 0.007440881337970495
Iter 53 loss: 0.020000970902766573
Change in x_s: 0.007188154384493828
Iter 54 loss: 0.020001009093528466
Change in x_s: 0.00695628160610795
Iter 55 loss: 0.020001044367658753
Change in x_s: 0.00674373097717762
Iter 56 loss: 0.020001075240973644
Change in x_s: 0.006548846140503883
Iter 57 loss: 0.020001105654369338
Change in x_s: 0.006369963753968477
Iter 58 loss: 0.020001137804211366
Change in x_s: 0.006205426529049873
Iter 59 loss

### Group regularized Results

In [47]:
from src.optimal_transport import group_feature_transport
import pickle

for i in range(3):
    np.random.seed(i * 1000)
    x_s_group_train = group_feature_transport(adult_source[subset_train, :], adult_target[subset_train, :], adult_source_feasible_groups[subset_train, :], adult_target_feasible_groups[subset_train, :], 36, 0.8, 100) # For adult feasibility split male/female
    x_s_group_test = transform_samples(adult_source[subset_test, :], adult_source[subset_train, :], x_s_group_train)

    total, worst = group_percent_explained(
        adult_source[subset_train, :],
        adult_scaler.transform(adult_scaler.inverse_transform(x_s_group_train).round(0)),
        adult_target[subset_train, :],
        adult_source_feasible_groups[subset_train, :],
        adult_target_feasible_groups[subset_train, :],
        adult_feasible_names)
    group_total.append(total)
    group_worst.append(worst)
    
    pf, pf_worst = percent_flipped(lambda x: torch.from_numpy(clf.predict(x)), adult_source[subset_train, :], x_s_group_train, adult_source_feasible_groups[subset_train, :])
    group_pf.append(pf)
    group_pf_worst.append(pf_worst)

    feasible_train = is_adult_feasible_change(
        adult_scaler.inverse_transform(adult_source[subset_train, :]).round(0),
        adult_scaler.inverse_transform(x_s_group_train).round(0))
    group_feas.append(np.sum(feasible_train) / feasible_train.shape[0])

[0, 1]
Iter 0 loss: 1.0
Change in x_s: 26.418384552001953
Iter 1 loss: 0.45676147716507765
Change in x_s: 19.047012329101562
Iter 2 loss: 0.2657473880617148
Change in x_s: 13.913678169250488
Iter 3 loss: 0.19112348111892047
Change in x_s: 10.707784652709961
Iter 4 loss: 0.15705122253307252
Change in x_s: 8.410999298095703
Iter 5 loss: 0.13904899217047967
Change in x_s: 6.771783828735352
Iter 6 loss: 0.12799568439841463
Change in x_s: 5.511378765106201
Iter 7 loss: 0.12043568850523137
Change in x_s: 4.451881408691406
Iter 8 loss: 0.11590059282080006
Change in x_s: 3.575915813446045
Iter 9 loss: 0.11265913440220521
Change in x_s: 2.934279680252075
Iter 10 loss: 0.1103958386343096
Change in x_s: 2.376434326171875
Iter 11 loss: 0.10886159044398899
Change in x_s: 1.9646152257919312
Iter 12 loss: 0.10777423539945977
Change in x_s: 1.6932913064956665
Iter 13 loss: 0.1068037077810092
Change in x_s: 1.3799316883087158
Iter 14 loss: 0.10616643318223701
Change in x_s: 1.171427607536316
Iter 15 lo

Iter 19 loss: 0.10903187805427789
Change in x_s: 0.6080107688903809
Iter 20 loss: 0.1088903521460102
Change in x_s: 0.5844631195068359
Iter 21 loss: 0.10896856480798336
Change in x_s: 0.5855924487113953
Iter 22 loss: 0.10880941691887917
Change in x_s: 0.5398333668708801
Iter 23 loss: 0.10889638151382014
Change in x_s: 0.5577247738838196
Iter 24 loss: 0.10866661630234103
Change in x_s: 0.6618082523345947
Iter 25 loss: 0.10846078370812837
Change in x_s: 0.578478217124939
Iter 26 loss: 0.10825329223100238
Change in x_s: 0.5222992300987244
Iter 27 loss: 0.10833353272800744
Change in x_s: 0.5145963430404663
Iter 28 loss: 0.10823070419780699
Change in x_s: 0.5153884887695312
Iter 29 loss: 0.10831509365285552
Change in x_s: 0.5159770846366882
Iter 30 loss: 0.10819647634887453
Change in x_s: 0.5112419128417969
Iter 31 loss: 0.10828272573034128
Change in x_s: 0.5080833435058594
Iter 32 loss: 0.10818331313057444
Change in x_s: 0.5051348805427551
Iter 33 loss: 0.10828248710706379
Change in x_s: 0

Iter 38 loss: 0.10540067474961526
Change in x_s: 0.47136738896369934
Iter 39 loss: 0.10553732022864197
Change in x_s: 0.45578882098197937
Iter 40 loss: 0.10539781874201704
Change in x_s: 0.4424665570259094
Iter 41 loss: 0.10551692852882998
Change in x_s: 0.4547671973705292
Iter 42 loss: 0.105404966587427
Change in x_s: 0.47399279475212097
Iter 43 loss: 0.1055411925657806
Change in x_s: 0.45826801657676697
Iter 44 loss: 0.10540375315279867
Change in x_s: 0.4447600245475769
Iter 45 loss: 0.10551979019708757
Change in x_s: 0.45687335729599
Iter 46 loss: 0.10541143325617763
Change in x_s: 0.47677284479141235
Iter 47 loss: 0.10555070585161058
Change in x_s: 0.4609135687351227
Iter 48 loss: 0.10540506628154048
Change in x_s: 0.44705238938331604
Iter 49 loss: 0.10552864229709977
Change in x_s: 0.45883873105049133
Iter 50 loss: 0.1054184510768981
Change in x_s: 0.4792109429836273
Iter 51 loss: 0.10556001142552512
Change in x_s: 0.46320003271102905
Iter 52 loss: 0.10541230537089837
Change in x_

In [48]:
print(np.mean(reg_total), np.std(reg_total))
print(np.mean(reg_worst), np.std(reg_worst))
print(np.mean(reg_feas), np.std(reg_feas))
print(np.mean(reg_pf), np.std(reg_pf))
print(np.mean(reg_pf_worst), np.std(reg_pf_worst))
print()
print(np.mean(group_total), np.std(group_total))
print(np.mean(group_worst), np.std(group_worst))
print(np.mean(group_feas), np.std(group_feas))
print(np.mean(group_pf), np.std(group_pf))
print(np.mean(group_pf_worst), np.std(group_pf_worst))

95.56419420792899 0.24898799271534483
61.231288781967635 1.0601979884330797
0.8099999999999999 0.003600411499115458
53.611111111111114 0.39283710065919086
51.822916666666664 0.5126307209643106

76.54896577336733 0.19159903713143472
81.20397160577384 0.40825508531068017
1.0 0.0
50.88888888888889 0.2832788618662674
48.567708333333336 0.5126307209643106


## Breast Cancer

### Non-group regularized results

In [49]:
from sklearn.ensemble import GradientBoostingClassifier

X_train = np.concatenate([bc_source, bc_target])
y_train = np.concatenate([bc_source_labels[:, 0], bc_target_labels[:, 0]])
clf = GradientBoostingClassifier(n_estimators=100).fit(X_train, y_train)

In [50]:
from src.optimal_transport import group_feature_transport, transform_samples
import pickle

reg_total = []
reg_worst = []
reg_feas = []
reg_pf = []
reg_pf_worst = []
group_total = []
group_worst = []
group_feas = []
group_pf = []
group_pf_worst = []
for i in range(3):
    np.random.seed(i)
    no_group_s = np.ones((bc_source.shape[0], 1))
    no_group_t = np.ones((bc_target.shape[0], 1))
    x_s = group_feature_transport(bc_source, bc_target, no_group_s, no_group_t, 36, 0.1, 100)

    total, worst = group_percent_explained(
        bc_source,
        x_s,
        bc_target,
        bc_source_feasible_groups,
        bc_target_feasible_groups,
        bc_feasible_names)
    reg_total.append(total)
    reg_worst.append(worst)

    pf, pf_worst = percent_flipped(lambda x: torch.from_numpy(clf.predict(x)), bc_source, x_s, bc_source_feasible_groups)
    reg_pf.append(pf)
    reg_pf_worst.append(pf_worst)

    feasible = is_bc_feasible_change(bc_source, x_s)
    reg_feas.append(np.sum(feasible) / feasible.shape[0])

[0]
Iter 0 loss: 1.0
Change in x_s: 0.7430683970451355
Iter 1 loss: 0.9743287155434428
Change in x_s: 0.7357691526412964
Iter 2 loss: 0.9492137342714039
Change in x_s: 0.727379322052002
Iter 3 loss: 0.9246912308179767
Change in x_s: 0.7186005115509033
Iter 4 loss: 0.9007684586660815
Change in x_s: 0.7096800804138184
Iter 5 loss: 0.8774421835778892
Change in x_s: 0.7007132768630981
Iter 6 loss: 0.8547049341634138
Change in x_s: 0.6918578147888184
Iter 7 loss: 0.8325471162217053
Change in x_s: 0.683045506477356
Iter 8 loss: 0.8109566792214992
Change in x_s: 0.6743687987327576
Iter 9 loss: 0.7899193815109584
Change in x_s: 0.6658291220664978
Iter 10 loss: 0.7694208376288689
Change in x_s: 0.6574004292488098
Iter 11 loss: 0.7494472700579882
Change in x_s: 0.6490066051483154
Iter 12 loss: 0.7299867382993771
Change in x_s: 0.6405799984931946
Iter 13 loss: 0.711028154253939
Change in x_s: 0.6322000026702881
Iter 14 loss: 0.6925625291799958
Change in x_s: 0.6239001154899597
Iter 15 loss: 0.674

Iter 17 loss: 0.6428365714798968
Change in x_s: 0.5943564176559448
Iter 18 loss: 0.6263021068123708
Change in x_s: 0.5866885185241699
Iter 19 loss: 0.6101918588761491
Change in x_s: 0.5791271328926086
Iter 20 loss: 0.5944943267916084
Change in x_s: 0.5716668963432312
Iter 21 loss: 0.5791998367388952
Change in x_s: 0.5642837882041931
Iter 22 loss: 0.564298027160154
Change in x_s: 0.5569865703582764
Iter 23 loss: 0.5497790742856984
Change in x_s: 0.5497786402702332
Iter 24 loss: 0.5356333073199062
Change in x_s: 0.5426625609397888
Iter 25 loss: 0.5218512038129688
Change in x_s: 0.5356390476226807
Iter 26 loss: 0.5084234270419186
Change in x_s: 0.5287074446678162
Iter 27 loss: 0.49534085671308264
Change in x_s: 0.5218669772148132
Iter 28 loss: 0.4825946038922538
Change in x_s: 0.5151169896125793
Iter 29 loss: 0.47017602670756425
Change in x_s: 0.5084563493728638
Iter 30 loss: 0.4580767254605803
Change in x_s: 0.5018836855888367
Iter 31 loss: 0.4462880472808201
Change in x_s: 0.49540358781

Iter 34 loss: 0.4118662881244451
Change in x_s: 0.4765527546405792
Iter 35 loss: 0.4012232707923179
Change in x_s: 0.4703609347343445
Iter 36 loss: 0.3908551244762782
Change in x_s: 0.46424758434295654
Iter 37 loss: 0.3807548044092713
Change in x_s: 0.4582122564315796
Iter 38 loss: 0.37091542913329767
Change in x_s: 0.4522545635700226
Iter 39 loss: 0.36133027980964233
Change in x_s: 0.4463737905025482
Iter 40 loss: 0.35199280235253916
Change in x_s: 0.44056910276412964
Iter 41 loss: 0.34289660686941414
Change in x_s: 0.4348396956920624
Iter 42 loss: 0.3340354638146278
Change in x_s: 0.42918461561203003
Iter 43 loss: 0.32540330376800614
Change in x_s: 0.42360299825668335
Iter 44 loss: 0.31699421164447433
Change in x_s: 0.41809386014938354
Iter 45 loss: 0.3088024251937295
Change in x_s: 0.41265636682510376
Iter 46 loss: 0.3008223296820746
Change in x_s: 0.40728965401649475
Iter 47 loss: 0.29304845409898744
Change in x_s: 0.40199264883995056
Iter 48 loss: 0.2854754698405369
Change in x_s:

### Group regularized Results

In [51]:
from src.optimal_transport import group_feature_transport
import pickle

for i in range(3):
    np.random.seed(i)
    x_s_group = group_feature_transport(bc_source,
                                        bc_target,
                                        bc_source_feasible_groups,
                                        bc_target_feasible_groups,
                                        36,
                                        0.1,
                                        100)  # For adult feasibility split male/female

    total, worst = group_percent_explained(
        bc_source,
        x_s_group,
        bc_target,
        bc_source_feasible_groups,
        bc_target_feasible_groups,
        bc_feasible_names)
    group_total.append(total)
    group_worst.append(worst)

    pf, pf_worst = percent_flipped(lambda x: torch.from_numpy(clf.predict(x)), bc_source, x_s_group, bc_source_feasible_groups)
    group_pf.append(pf)
    group_pf_worst.append(pf_worst)

    feasible = is_bc_feasible_change(bc_source, x_s_group)
    group_feas.append(np.sum(feasible) / feasible.shape[0])

[0, 1, 2]
Iter 0 loss: 1.0
Change in x_s: 0.7726396322250366
Iter 1 loss: 0.9721616604006733
Change in x_s: 0.7615392804145813
Iter 2 loss: 0.9451326311754189
Change in x_s: 0.7495743632316589
Iter 3 loss: 0.9189274050550461
Change in x_s: 0.737298846244812
Iter 4 loss: 0.8935397392847801
Change in x_s: 0.7252535223960876
Iter 5 loss: 0.8689528305999765
Change in x_s: 0.7136486172676086
Iter 6 loss: 0.8451410380479827
Change in x_s: 0.7023899555206299
Iter 7 loss: 0.822076470703967
Change in x_s: 0.6913402080535889
Iter 8 loss: 0.7997362075880226
Change in x_s: 0.6803524494171143
Iter 9 loss: 0.7781047954491258
Change in x_s: 0.6695005893707275
Iter 10 loss: 0.7571605640809969
Change in x_s: 0.6588816046714783
Iter 11 loss: 0.7368776872492152
Change in x_s: 0.6484884023666382
Iter 12 loss: 0.7172319331848711
Change in x_s: 0.6382676362991333
Iter 13 loss: 0.6982015090164765
Change in x_s: 0.6283029913902283
Iter 14 loss: 0.6797617107163758
Change in x_s: 0.6186479330062866
Iter 15 loss

Iter 17 loss: 0.6280812060498819
Change in x_s: 0.5901978611946106
Iter 18 loss: 0.6118073670848123
Change in x_s: 0.581364631652832
Iter 19 loss: 0.5960168836071992
Change in x_s: 0.5727890133857727
Iter 20 loss: 0.580689119844376
Change in x_s: 0.5643489360809326
Iter 21 loss: 0.5658105620974441
Change in x_s: 0.5560515522956848
Iter 22 loss: 0.5513682510147855
Change in x_s: 0.547931969165802
Iter 23 loss: 0.5373497418888187
Change in x_s: 0.5399172902107239
Iter 24 loss: 0.523743786255732
Change in x_s: 0.5320466756820679
Iter 25 loss: 0.5105363582152092
Change in x_s: 0.5243544578552246
Iter 26 loss: 0.49771331283767
Change in x_s: 0.5168423056602478
Iter 27 loss: 0.4852606389561353
Change in x_s: 0.509497880935669
Iter 28 loss: 0.473165864815137
Change in x_s: 0.5023123621940613
Iter 29 loss: 0.46141708095291967
Change in x_s: 0.49528244137763977
Iter 30 loss: 0.4500026194686897
Change in x_s: 0.488398015499115
Iter 31 loss: 0.438911318417449
Change in x_s: 0.48164549469947815
It

Iter 34 loss: 0.4070023774808701
Change in x_s: 0.4610429108142853
Iter 35 loss: 0.39713557323943527
Change in x_s: 0.4546419382095337
Iter 36 loss: 0.3875447948531633
Change in x_s: 0.4484018385410309
Iter 37 loss: 0.3782196899773871
Change in x_s: 0.4423160254955292
Iter 38 loss: 0.3691505423059537
Change in x_s: 0.4363647699356079
Iter 39 loss: 0.36032822763451716
Change in x_s: 0.43052592873573303
Iter 40 loss: 0.35174545165135707
Change in x_s: 0.4247801601886749
Iter 41 loss: 0.343394508039333
Change in x_s: 0.41911688446998596
Iter 42 loss: 0.3352684423257939
Change in x_s: 0.41352611780166626
Iter 43 loss: 0.32736075898716865
Change in x_s: 0.40800735354423523
Iter 44 loss: 0.31966519330112897
Change in x_s: 0.40257519483566284
Iter 45 loss: 0.31217509943333605
Change in x_s: 0.3972439467906952
Iter 46 loss: 0.30488426230577015
Change in x_s: 0.39201369881629944
Iter 47 loss: 0.29778603717512875
Change in x_s: 0.38688749074935913
Iter 48 loss: 0.29087416608895655
Change in x_s:

In [52]:
print(np.mean(reg_total), np.std(reg_total))
print(np.mean(reg_worst), np.std(reg_worst))
print(np.mean(reg_feas), np.std(reg_feas))
print(np.mean(reg_pf), np.std(reg_pf))
print(np.mean(group_total), np.std(group_total))
print(np.mean(group_worst), np.std(group_worst))
print(np.mean(group_feas), np.std(group_feas))
print(np.mean(group_pf), np.std(group_pf))

93.10009879105264 0.05826983617022046
85.79442970892954 0.12327355228204803
0.669811320754717 0.0066708186904391
8.018867924528303 0.667081869043913
90.54350091071701 0.02432273858536622
89.10380082205059 0.06765719856153707
0.6147798742138365 0.012380515525175792
9.59119496855346 0.22236062301463722
